# Baseline Traduccion Español 
Usando Slides

## Install and data

In [6]:
from google.colab import drive
import os

# Montar Google Drive
drive.mount('/content/drive')

# Rutas de los archivos zip en Google Drive
test_zip_path = '/content/drive/MyDrive/test.zip'
test_lst = '/content/drive/MyDrive/test.lst'

# Directorio de destino en Colab
colab_content_path = '/content/'

# Copiar los archivos zip a Colab
!cp "{test_zip_path}" "{colab_content_path}"
!cp "{test_lst}" "{colab_content_path}"

print(f"Archivos copiados a {colab_content_path}")
print(os.listdir(colab_content_path))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Archivos copiados a /content/
['.config', 'drive', 'test', 'test.lst', 'test.zip', 'sample_data']


In [7]:
!unzip /content/test.zip -d /content/test

Archive:  /content/test.zip
replace /content/test/test.lst? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [8]:
! pip install  backoff  transformers==4.49
! pip install sacrebleu unbabel-comet

In [9]:
# ────────────────────────────── 1. Dependencias de SO ─────────────────────────
!apt-get update -qq
!apt-get install -y poppler-utils libreoffice

# ────────────────────────────── 2. Paquetes Python ────────────────────────────
!pip install pdf2image pillow tqdm   # pdf2image necesita Poppler

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  apparmor default-jre default-jre-headless dictionaries-common
  firebird3.0-common firebird3.0-common-doc firebird3.0-server-core
  firebird3.0-utils fonts-crosextra-caladea fonts-crosextra-carlito
  fonts-dejavu fonts-dejavu-core fonts-dejavu-extra fonts-liberation2
  fonts-linuxlibertine fonts-noto-core fonts-noto-extra fonts-noto-mono
  fonts-noto-ui-core fonts-opensymbol fonts-sil-gentium
  fonts-sil-gentium-basic gstreamer1.0-gl gstreamer1.0-gtk3 hunspell-en-us
  libabsl20210324 libabw-0.1-1 libatk-wrapper-java libatk-wrapper-java-jni
  libbsh-java libcdr-0.1-1 libclucene-contribs1v5 libclucene-core1v5
  libcolamd2 libe-book-0.1-1 libel-api-java 

## Load

### Load Test data

In [ ]:
from pathlib import Path
import subprocess, soundfile as sf, datetime as dt

FFMPEG = "ffmpeg"          # o la ruta absoluta si no está en PATH

def m4a_to_wav(path_m4a, sr_out=16_000):
    """Convierte 1 × .m4a → .wav (mono, 16 kHz) y devuelve la ruta del WAV."""
    path_m4a = Path(path_m4a)
    wav_path = path_m4a.with_suffix(".wav")
    subprocess.run(
        [FFMPEG, "-loglevel", "error", "-y", "-i", str(path_m4a),
         "-ac", "1", "-ar", str(sr_out), str(wav_path)],
        check=True
    )
    return wav_path


def get_files(folder='test'):
  with open(f'/content/{folder}/{folder}.lst', 'r', encoding='utf-8') as f:
    files = f.read()
    files = files.splitlines()
    print(files)
    return files

In [2]:
from pathlib import Path
import tempfile, subprocess
from typing import List, Tuple
from PIL import Image
from pdf2image import convert_from_path

def _ppt_to_pdf(src: Path) -> Path:
    tmp = Path(tempfile.mkdtemp())
    pdf_out = tmp / f"{src.stem}.pdf"
    cmd = ["soffice", "--headless", "--convert-to", "pdf", "--outdir", str(tmp), str(src)]
    subprocess.run(cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    return pdf_out  # se borrará cuando el tmp sea recolectado por el sistema

def _ensure_pdf(path: str | Path) -> Path:
    extensions = {".pdf", ".ppt", ".pptx"}
    ext = None
    for e in extensions:
        if Path(f"{path}{e}").exists():
            ext = e
            break
    p=Path(f"{path}{ext}")
    if ext == ".pdf":
        return p
    if ext in {".ppt", ".pptx"}:
        return _ppt_to_pdf(p)
    raise ValueError(f"Formato no soportado: {ext}. Usa .pdf/.ppt/.pptx")

# 2) Escribe PNGs al disco (menos RAM) y devuelve sus rutas
def _natural_sort_key(p: Path):
    # Ordena "page-2.png" < "page-10.png" de forma numérica
    stem = p.stem  # e.g., "page-12"
    try:
        n = int(stem.split("-")[-1])
    except Exception:
        n = 10**9  # si no hay número, mándalo al final
    return (stem.split("-")[0], n)

def slides_to_png_paths_resized(
    file_path: str | Path,
    out_dir: str | Path,
    size: Tuple[int, int] = (1600, 900),
    reuse_if_exists: bool = True  # ← controla si reutilizamos lo existente
) -> List[str]:
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    # Si ya hay PNGs y queremos reutilizar, devolverlos
    if reuse_if_exists:
        existing = sorted(out_dir.glob("*.png"), key=_natural_sort_key)
        if existing:
            return [str(p) for p in existing]
    pdf = _ensure_pdf(file_path)
    # Guardar directamente los PNG ya redimensionados
    paths = convert_from_path(pdf, size=size, fmt="png", output_folder=str(out_dir), paths_only=True)
    # pdf2image genera nombres tipo page-1.png; los ordenamos
    return [str(p) for p in sorted(map(Path, paths))]


In [3]:
from pathlib import Path
from typing import List, Dict, Tuple, Optional

def _ms_to_seconds(hms: str) -> float:
    m, s, _ = hms.split(":")
    return int(m)*60 + int(s)

def parse_slide_blocks(path: str | Path) -> Dict[str, Dict]:
    """
    Lee un archivo con múltiples bloques:
      <ID> <URL>
      <slide_id|None> <HH:MM:SS>
      ...
    Devuelve dict: { id: {"url": str, "timeline": [(t_sec, slide_id|None), ...]} }
    """
    p = Path(path)
    blocks: Dict[str, Dict] = {}
    cur_id = None
    for line in p.read_text().splitlines():
        line = line.strip()
        if not line:
            continue

        parts = line.split()
        # Cabecera: "<ID> <URL>"
        if len(parts) == 2 and parts[1].startswith("http"):
            cur_id = parts[0]
            blocks[cur_id] = []
            continue

        # Línea de cambio: "<slide_id|None> <HH:MM:SS>"
        if cur_id is not None and len(parts) == 2 and ":" in parts[1]:
            slide_raw, hms = parts
            slide_id = None if slide_raw == "None" else int(slide_raw)
            t = _ms_to_seconds(hms)
            blocks[cur_id].append((t, slide_id))

    # Ordena por tiempo y asegura que hay un evento en t=0
    for k,v in blocks.items():
        v.sort(key=lambda x: x[0])

    return blocks

def build_slide_intervals(timeline: List[Tuple[float, Optional[int]]]) -> List[Dict]:
    """
    Convierte una timeline en intervalos contiguos: [{"start":t0,"end":t1,"slide":id}, ...]
    El último intervalo termina en +inf (None).
    """
    out = []
    for i, (t, slide) in enumerate(timeline):
        t_next = timeline[i+1][0] if i+1 < len(timeline) else None
        out.append({"start": t, "end": t_next, "slide": slide})
    return out


In [4]:
def slide_at(time_s: float, intervals: List[Dict]) -> Optional[int]:
    """Devuelve la slide activa en 'time_s'."""
    # búsqueda lineal (puedes optimizar con bisect si quieres)
    for seg in intervals:
        if seg["end"] is None:  # último intervalo
            return seg["slide"]
        if seg["start"] <= time_s < seg["end"]:
            return seg["slide"]
    return None

from typing import List, Optional

def slides_in_interval(
    start: float,
    end: float,
    intervals: List[dict],
    *,
    min_fraction: float = 0.10,   # % mínimo del segmento que debe cubrir la slide
    min_seconds: float = 0.0,     # y/o segundos absolutos mínimos de cobertura
    slack: float = 0.25,          # margen (±s) para tolerar pequeñas imprecisiones
    sort_by_coverage: bool = True,
    top_k: Optional[int] = 1
) -> List[int]:
    """
    Devuelve las slides que realmente 'cuentan' en [start, end), filtrando
    las que cubren menos de min_fraction del segmento o menos de min_seconds.

    - 'slack' expande el segmento a [start-slack, end+slack] para evitar cortar
      por milésimas (timings manuales, FPS, etc.).
    - Si sort_by_coverage=True, ordena por mayor cobertura; si no, por aparición.
    - 'top_k' limita el nº de slides devueltas (tras el filtro).
    """
    seg_len = max(1e-6, end - start)  # evita división por cero
    seg_lo = start - slack
    seg_hi = end + slack

    cover_by_slide = {}   # slide_id -> (cover_seconds, first_order)

    order = 0
    for seg in intervals:
        slide = seg["slide"]
        if slide is None:
            continue

        s_lo = seg["start"]
        s_hi = float("inf") if seg["end"] is None else seg["end"]

        # intersección con margen
        lo = max(seg_lo, s_lo)
        hi = min(seg_hi, s_hi)
        overlap = max(0.0, hi - lo)
        if overlap <= 0:
            continue

        if slide not in cover_by_slide:
            cover_by_slide[slide] = [0.0, order]
            order += 1
        cover_by_slide[slide][0] += overlap

    # Filtro por cobertura
    out = []
    for slide, (cover_s, first_order) in cover_by_slide.items():
        frac = cover_s / seg_len
        if cover_s >= min_seconds or frac >= min_fraction:
            out.append((slide, cover_s, first_order))

    if not out:
        return []

    # Orden
    if sort_by_coverage:
        out.sort(key=lambda x: (-x[1], x[2]))   # más cobertura primero
    else:
        out.sort(key=lambda x: x[2])            # por aparición

    if top_k is not None:
        out = out[:top_k]

    return [slide for slide, _, _ in out]


In [5]:
from pathlib import Path
import datasets as ds


def build_dataset(folder="train", language="es", files=None,
                  dpi=150, target_size=(1600,900), mode="letterbox",
                  bg="white", store_slide_paths=False):
    rows = []
    files = get_files(folder) if files is None else files

    # Carga y parsea el maestro (una vez)
    blocks = parse_slide_blocks(f'{folder}.lst')

    for name in files:
        base = Path(f"/content/{folder}/{name}/manual_translations/{language}")
        with open(base / f"{name}.lst") as f:
            times = [(float(a), float(b)) for a,b in (ln.split() for ln in f.read().splitlines())]
        with open(base / f"{name}.en") as f:
            src = f.read().splitlines()
        with open(base / f"{name}.{language}") as f:
            tgt = f.read().splitlines()
        assert len(times) == len(src) == len(tgt)

        # 1) Preprocesa slides una vez (o usa caché si ya existen)
        slides_root = Path(f"/content/{folder}/{name}/slides")
        paths=slides_to_png_paths_resized(slides_root,f'/content/{folder}/{name}/slides/')
        slide_paths = [str(Path(p).relative_to(slides_root)) for p in paths]
        print(paths)
        print(slide_paths)

        # 2) Audio (como ya hacías)
        wav_path = str(m4a_to_wav(f"/content/{folder}/{name}/{name}.m4a"))

        if str(name) in blocks:
            intervals = build_slide_intervals(blocks[str(name)])
        else:
            intervals = [{"start":0.0,"end":None,"slide":None}]
            print(f"[WARN] No hay bloque de slides para {name}")

        # 3) Crea filas por segmento reusando las mismas referencias a slides
        for (s, e), txt_in, txt_out in zip(times, src, tgt):
            slides  = slides_in_interval(s, e, intervals)
            print(slides)
            row = {
                "wav_path": wav_path,
                "start": s,
                "end": e,
                "source_text": txt_in,
                "target_text": txt_out,
                "slide_paths": paths[slides[0]] if slides else None,

            }
            rows.append(row)

    return ds.Dataset.from_list(rows)
test_ds   = build_dataset("test")


['404', '596001', '605000', '606', '545006']
['/content/test/404/slides/91d1c214-320c-4e06-a5be-b9f790377697-01.png', '/content/test/404/slides/91d1c214-320c-4e06-a5be-b9f790377697-02.png', '/content/test/404/slides/91d1c214-320c-4e06-a5be-b9f790377697-03.png', '/content/test/404/slides/91d1c214-320c-4e06-a5be-b9f790377697-04.png', '/content/test/404/slides/91d1c214-320c-4e06-a5be-b9f790377697-05.png', '/content/test/404/slides/91d1c214-320c-4e06-a5be-b9f790377697-06.png', '/content/test/404/slides/91d1c214-320c-4e06-a5be-b9f790377697-07.png', '/content/test/404/slides/91d1c214-320c-4e06-a5be-b9f790377697-08.png', '/content/test/404/slides/91d1c214-320c-4e06-a5be-b9f790377697-09.png', '/content/test/404/slides/91d1c214-320c-4e06-a5be-b9f790377697-10.png', '/content/test/404/slides/91d1c214-320c-4e06-a5be-b9f790377697-11.png', '/content/test/404/slides/91d1c214-320c-4e06-a5be-b9f790377697-12.png', '/content/test/404/slides/91d1c214-320c-4e06-a5be-b9f790377697-13.png', '/content/test/404

In [6]:
test_ds[0]

{'wav_path': '/content/test/404/404.wav',
 'start': 0.0,
 'end': 2.13,
 'source_text': 'Good morning, afternoon and evening.',
 'target_text': 'Buenos días, buenas tardes, buenas noches.',
 'slide_paths': '/content/test/404/slides/91d1c214-320c-4e06-a5be-b9f790377697-02.png'}

### Load Model

In [7]:
from transformers import AutoModelForCausalLM, AutoProcessor, GenerationConfig

import torch

model_name = 'microsoft/Phi-4-multimodal-instruct'
processor = AutoProcessor.from_pretrained(model_name,trust_remote_code = True)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
    _attn_implementation='sdpa',
)

generation_config = GenerationConfig.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/auto/image_processing_auto.py:594: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow 

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

## Basic Inference

In [20]:

import torch, soundfile as sf
from PIL import Image

device = torch.device("cuda")

def load_audio_segment(wav_path, start, end, sr=16_000):
    # lee sólo el fragmento que necesitas
    with sf.SoundFile(wav_path) as f:
        f.seek(int(start*sr))
        audio = f.read(int((end-start)*sr), dtype="float32")
    return audio, sr

def build_prompt(task_text="Traduce el audio al Español."):
    # Prompt de chat con placeholders multimodales
    messages = [
        {"role":"user",
         "content": "<|audio_1|>\n<|image_1|>\n" + task_text}
    ]
    return processor.tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )

def make_inputs(row, text):
    # row: un registro de tu dataset con 'wav_path', 'start','end','slides'
    audio = load_audio_segment(row["wav_path"], row["start"], row["end"])  # (np.array, sr)
    # carga la slide
    image = Image.open(row['slide_paths']).convert("RGB")

    prompt = build_prompt(text)
    inputs = processor(
        text=[prompt],
        audios=[audio],             # lista de (waveform, sr)
        images=[image],             # lista de PIL Images
        return_tensors="pt",
        padding=True
    )
    return inputs.to(device)


In [36]:
model.eval()
model.gradient_checkpointing_disable()   # evita warnings/bugs en eval
with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
    inputs=make_inputs(test_ds[0], text="Translate the audio to ES-es.")
    gen_ids = model.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=False,
        num_beams=1,
        use_cache=True,
        num_logits_to_keep=0
    )
# cortar el prompt textual
input_len = inputs["input_ids"].shape[1]
decoded = processor.batch_decode(gen_ids[:, input_len:].cpu(), skip_special_tokens=True)
print(decoded[0])

Good morning, afternoon, and evening.


In [37]:
model.eval()
model.gradient_checkpointing_disable()   # evita warnings/bugs en eval
with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
    inputs=make_inputs(test_ds[1], text="Translate the audio to ES-es.")
    gen_ids = model.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=False,
        num_beams=1,
        use_cache=True,
        num_logits_to_keep=0
    )
# cortar el prompt textual
input_len = inputs["input_ids"].shape[1]
decoded = processor.batch_decode(gen_ids[:, input_len:].cpu(), skip_special_tokens=True)
print(decoded[0])

Esta es la presentación del conferencero.


In [ ]:
model.gradient_checkpointing_disable()   # evita warnings/bugs en eval
with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
    inputs=make_inputs(test_ds[1], text="Use the image to disambiguate terminology. Translate the audio to  ES-es.")
    gen_ids = model.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=False,
        num_beams=1,
        use_cache=True,
        num_logits_to_keep=0
    )
# cortar el prompt textual
input_len = inputs["input_ids"].shape[1]
decoded = processor.batch_decode(gen_ids[:, input_len:].cpu(), skip_special_tokens=True)
print(decoded[0])


Esta es la presentación del conferenciante
e-sesion 404
The impact of oncological surgery on the outcomes
Expert: Prof. Charles M. Balch, University of Texas MD Anderson Cancer Center, Houston, USA
Discusant: Prof. Riccardo Audisio, Whiston Hospital, University of Liverpool, Prescot, UK
Extract from the e-ESO policy
The website contains presentations aimed at providing new knowledge and competences, and is intended as an informational and
educational tool mainly designed for oncology professionals and other physicians interested in oncology.
These materials remain property of the authors or ESO respectively.
ESO is not responsible for any injury and/or damage to persons or property as a matter of a products liability, negligence or otherwise, or from any use or operation of any methods, products, instructions or ideas contained in the materials published in these
presentations. Because of the rapid advances in medical sciences, we recommend that independent verification of diagnoses a

In [31]:
model.gradient_checkpointing_disable()   # evita warnings/bugs en eval
with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
    inputs=make_inputs(test_ds[1], text="Traduce el audio a ES-es.")
    gen_ids = model.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=False,
        num_beams=1,
        use_cache=True,
        num_logits_to_keep=0
    )
# cortar el prompt textual
input_len = inputs["input_ids"].shape[1]
decoded = processor.batch_decode(gen_ids[:, input_len:].cpu(), skip_special_tokens=True)
print(decoded[0])

Esta es la presentación del operador de conferencia.


In [ ]:
model.gradient_checkpointing_disable()   # evita warnings/bugs en eval
with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
    inputs=make_inputs(test_ds[10], text="Using the attached image(s) as context, disambiguate acronyms and technical terms. Translate the spoken English audio into Spanish (es-ES). Preserve numbers, units, drug names, and abbreviations; keep slide titles and proper names as in the source.")
    gen_ids = model.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=False,
        num_beams=1,
        use_cache=True,
        num_logits_to_keep=0
    )
# cortar el prompt textual
input_len = inputs["input_ids"].shape[1]
decoded = processor.batch_decode(gen_ids[:, input_len:].cpu(), skip_special_tokens=True)
print(decoded[0])

The image shows a slide from an e-session, specifically session number 404, which is focused on the impact of oncological surgery on outcomes. The slide features the logos of e-ESO and e-ES, indicating that this session is part of an online educational series. The session is presented by an expert, Prof. Charles M. Balch from the University of Texas MD Anderson Cancer Center, Houston, USA, and a discussant, Prof. Riccardo Audisio from Whiston Hospital, University of Liverpool, Prescot, UK. The slide also includes a disclaimer from the e-ESO policy, stating that the website provides new knowledge and competences for oncology professionals and is intended as an informational and educational tool. It emphasizes that the materials remain the property of the authors and that independent verification of diagnoses and drug dosages is recommended. The disclaimer also notes that e-ESO does not endorse any opinions expressed in the presentations and advises patients and the general public to see

In [34]:
model.gradient_checkpointing_disable()   # evita warnings/bugs en eval
with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
    inputs=make_inputs(test_ds[0], text="Traduce el audio a ES-es.")
    gen_ids = model.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=False,
        num_beams=1,
        use_cache=True,
        num_logits_to_keep=0
    )
# cortar el prompt textual
input_len = inputs["input_ids"].shape[1]
decoded = processor.batch_decode(gen_ids[:, input_len:].cpu(), skip_special_tokens=True)
print(decoded[0])

Good morning, afternoon, and evening.


In [35]:
model.gradient_checkpointing_disable()   # evita warnings/bugs en eval
with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
    inputs=make_inputs(test_ds[0], text="Describe the image")
    gen_ids = model.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=False,
        num_beams=1,
        use_cache=True,
        num_logits_to_keep=0
    )
# cortar el prompt textual
input_len = inputs["input_ids"].shape[1]
decoded = processor.batch_decode(gen_ids[:, input_len:].cpu(), skip_special_tokens=True)
print(decoded[0])

The image is a slide from an e-session, specifically e-session 404, titled 'The impact of oncological surgery on the outcomes'. It features the logos of e-ESO and e-ESB at the top right corner. The slide lists the expert and discussant for the session, Prof. Charles M. Balch from the University of Texas MD Anderson Cancer Center, Houston, USA, and Prof. Riccardo Audisio from Whiston Hospital, University of Liverpool, Prescot, UK. Below this, there is a disclaimer extract from the e-ESO policy, which outlines the purpose of the website, the responsibility of the authors and e-ESO, and a recommendation for independent verification of medical information. It also advises that patients and the general public should seek professional medical advice and notes that e-ESO does not endorse any opinions expressed in the presentations. At the bottom, there is a prompt to share the e-ESO experience using the hashtag #e_ESO. The slide number is indicated as 2.


## Baseline

### Run

In [8]:
import soundfile as sf
from collections import defaultdict

_open_wavs = {}                  # caché: path → SoundFile

def read_segment(path, start, end, sr=16_000):
    """Lee [start,end] s de un WAV grande sin cargarlo entero."""
    if path not in _open_wavs:
        _open_wavs[path] = sf.SoundFile(path)
    f = _open_wavs[path]
    frames = int((end - start) * sr)
    f.seek(int(start * sr))
    return f.read(frames, dtype="float32")

def segments_by_audio(ds):
    groups = defaultdict(list)
    for ex in ds:
        groups[ex["wav_path"]].append(ex)
    # ordenamos cada audio por inicio temporal
    for segs in groups.values():
        segs.sort(key=lambda x: x["start"])
    return groups


def build_prompt(task_text):
    # Prompt de chat con placeholders multimodales
    messages = [
        {"role":"user",
         "content": "<|audio_1|>\n<|image_1|>\n" + task_text}
    ]
    return processor.tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )



In [19]:
groups.keys()


dict_keys(['/content/test/596001/596001.wav', '/content/test/605000/605000.wav', '/content/test/606/606.wav', '/content/test/545006/545006.wav'])

In [28]:
from PIL import Image

def build_prompt_mm(task_text: str, use_image: bool):
    # Prompt de chat con placeholders multimodales
    content = "<|audio_1|>\n"
    if use_image:
        content += "<|image_1|>\n"
    content += task_text
    return processor.tokenizer.apply_chat_template(
        [{"role":"user","content": content}],
        tokenize=False, add_generation_prompt=True
    )

prompt_img   = build_prompt_mm("Translate the audio to es-ES.", use_image=True)
prompt_noimg = build_prompt_mm("Translate the audio to es-ES.", use_image=False)

def generate_output(groups):
    hyps_all, refs_all, srcs_all = [], [], []

    for _, segs in tqdm(groups.items(), desc="Audios"):
        hyps, refs, srcs = [], [], []

        for i in range(0, len(segs), batch_size):
            chunk = segs[i:i+batch_size]

            idx_with_img = [j for j,r in enumerate(chunk) if r.get("slide_paths")]
            idx_no_img   = [j for j,r in enumerate(chunk) if not r.get("slide_paths")]

            hyps_chunk = [None] * len(chunk)

            def run_group(idxs, use_image: bool):
                if not idxs:
                    return
                auds, imgs, prompts = [], [], []
                for j in idxs:
                    row = chunk[j]
                    auds.append((read_segment(row["wav_path"], row["start"], row["end"]), 16_000))
                    if use_image:
                        img = Image.open(row["slide_paths"]).convert("RGB")
                        imgs.append(img)
                        prompts.append(prompt_img)
                    else:
                        prompts.append(prompt_noimg)

                kwargs = dict(text=prompts, audios=auds, return_tensors="pt", padding=True)
                if use_image:
                    kwargs["images"] = imgs

                inputs = processor(**kwargs).to(device)

                with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
                    gen_ids = model.generate(
                        **inputs,
                        generation_config=generation_config,
                        max_new_tokens=1000,
                        num_logits_to_keep=0,
                    )

                # cortar prompt
                out = processor.batch_decode(
                    gen_ids[:, inputs["input_ids"].shape[1]:].cpu(),
                    skip_special_tokens=True,
                    clean_up_tokenization_spaces=False,
                )
                for k, j in enumerate(idxs):
                    hyps_chunk[j] = out[k]

                # limpieza
                if use_image:
                    for im in imgs: im.close()
                del inputs, gen_ids, auds, imgs, prompts
                torch.cuda.empty_cache(); gc.collect()

            run_group(idx_with_img, True)
            run_group(idx_no_img, False)

            hyps.extend(hyps_chunk)
            refs.extend([r["target_text"] for r in chunk])
            srcs.extend([r["source_text"] for r in chunk])

        print(hyps)
        print(refs)
        print(srcs)
        hyps_all.append(hyps); refs_all.append(refs); srcs_all.append(srcs)

    return hyps_all, refs_all, srcs_all


In [21]:
from __future__ import annotations

from itertools import chain
from typing import Callable, Dict, List, Tuple

import sacrebleu
from comet import download_model, load_from_checkpoint

# ────────────────────────────────────────────────────────────────────────────────
#  Cargamos (y cacheamos) el modelo COMET‑22 una sola vez
# ────────────────────────────────────────────────────────────────────────────────
_COMET_MODEL = None
_COMET_PATH = None
_MODEL_NAME = "Unbabel/wmt22-comet-da"


def _get_comet_model(gpus: int = 0):
    """Devuelve un modelo COMET‑22 listo para `predict` (cacheado)."""
    global _COMET_MODEL, _COMET_PATH

    if _COMET_MODEL is None:
        _COMET_PATH = download_model(_MODEL_NAME)  # se guarda en ~/.cache
        _COMET_MODEL = load_from_checkpoint(_COMET_PATH)

    # *Nota*: el parámetro `gpus` se pasa a `.predict` y **no** aquí, pero
    # exponemos el arg para quien quiera forzar CPU en la firma pública.
    return _COMET_MODEL


# ────────────────────────────────────────────────────────────────────────────────
#  Función principal
# ────────────────────────────────────────────────────────────────────────────────

def bleu_comet_by_audio(
    refs_audio: List[List[str]],
    hyps_audio: List[List[str]],
    srcs_audio: List[List[str]],
    transform: Callable[[str], str] = lambda x: x,
    comet_gpus: int = 0,
    comet_batch_size: int = 8,
) -> Tuple[Dict[str, float], List[Dict[str, float]]]:
    """Calcula BLEU y COMET‑22, **global** y **por audio**.

    Parameters
    ----------
    refs_audio, hyps_audio, srcs_audio : list[list[str]]
        Listas anidadas con el mismo nº de audios y segmentos.
    transform : callable
        Función de normalización por frase (identidad por defecto).
    comet_gpus : int
        Nº de GPUs a usar en `model.predict` (0 ⇒ CPU).
    comet_batch_size : int
        Tamaño de lote para COMET (trade‑off velocidad / memoria).

    Returns
    -------
    (global_metrics, per_audio_metrics)
        global_metrics    = {"bleu": float, "comet22": float}
        per_audio_metrics = [
            {"audio_id": i, "bleu": float, "comet22": float},
            ...
        ]
    """

    # ── Comprobaciones básicas ──────────────────────────────────────────────
    assert len(refs_audio) == len(hyps_audio) == len(srcs_audio), (
        "refs, hyps y srcs deben tener la misma longitud (nº audios)"
    )

    per_audio: List[Dict[str, float]] = []

    # ── Recorremos audio por audio ──────────────────────────────────────────
    comet_model = _get_comet_model(gpus=comet_gpus)

    for idx, (ref_seg, hyp_seg, src_seg) in enumerate(
        zip(refs_audio, hyps_audio, srcs_audio)
    ):
        assert len(ref_seg) == len(hyp_seg) == len(src_seg), (
            f"El audio {idx} contiene diferente nº de segmentos"
        )

        ref_seg = [transform(r) for r in ref_seg]
        hyp_seg = [transform(h) for h in hyp_seg]
        src_seg = [transform(s) for s in src_seg]

        # ── BLEU corpus‑level para el audio ────────────────────────────────
        bleu_score = sacrebleu.corpus_bleu(hyp_seg, [ref_seg]).score

        # ── COMET‑22 ───────────────────────────────────────────────────────
        samples = [  # una entrada por segmento
            {"src": s, "mt": h, "ref": r}
            for s, h, r in zip(src_seg, hyp_seg, ref_seg)
        ]
        comet_out = comet_model.predict(
            samples,
            batch_size=comet_batch_size,
            gpus=comet_gpus,
            progress_bar=False,
        )
        comet_score = comet_out["system_score"]  # media ya calculada

        per_audio.append(
            {
                "audio_id": idx,
                "bleu": bleu_score,
                "comet22": comet_score,
            }
        )

    # ── Métricas globales ──────────────────────────────────────────────────
    refs_all = list(chain.from_iterable(refs_audio))
    hyps_all = list(chain.from_iterable(hyps_audio))
    srcs_all = list(chain.from_iterable(srcs_audio))

    bleu_global = sacrebleu.corpus_bleu(hyps_all, [refs_all]).score

    comet_samples = [
        {"src": s, "mt": h, "ref": r}
        for s, h, r in zip(srcs_all, hyps_all, refs_all)
    ]
    comet_global = comet_model.predict(
        comet_samples,
        batch_size=comet_batch_size,
        gpus=comet_gpus,
        progress_bar=False,
    )["system_score"]

    return {"bleu": bleu_global, "comet22": comet_global}, per_audio





In [31]:

hyps, refs, srcs = generate_output(groups)


Audios:   0%|          | 0/4 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
Audios: 100%|██████████| 4/4 [2:03:29<00:00, 1852.39s/it]


In [41]:
global_metrics, per_audio = bleu_comet_by_audio(refs, hyps, srcs,
                                                comet_gpus=1, comet_batch_size=8)

INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytor

### Results

In [42]:

print("BLEU  global :", global_metrics["bleu"])
print("COMET-22    :", global_metrics["comet22"])

BLEU  global : 4.868949511525707
COMET-22    : 0.6054309277029648


In [43]:
per_audio

[{'audio_id': 0, 'bleu': 7.496381870565138, 'comet22': 0.6496172168781992},
 {'audio_id': 1, 'bleu': 2.553672454797138, 'comet22': 0.5879960074649807},
 {'audio_id': 2, 'bleu': 5.93933643459905, 'comet22': 0.6432677645808302},
 {'audio_id': 3, 'bleu': 0.37184679825923267, 'comet22': 0.49710749971623325},
 {'audio_id': 4, 'bleu': 11.621852061209827, 'comet22': 0.5989050285582163}]

In [ ]:
aux=[['Good morning, afternoon, and evening.', 'Esta es la conferencia del operador\ne-sesion 404\nThe impact of oncológico surgery on the outcomes\nExpert: Prof. Charles M. Balch, University of Texas MD Anderson Cancer Center, Houston, USA\nDiscusant: Prof. Riccardo Audisio, Whiston Hospital, University of Liverpool, Prescot, UK\nExtract from the e-ESO policy\nThe website contains presentations aimed at providing new knowledge and competences, and is intended as an informational and educational tool mainly designed for oncology professionals and other physicians interested in oncology.\nThese materials remain property of the authors or ESO respectively.\nESO is not responsible for any injury and/or damage to persons or property as a matter of a products liability, negligence or otherwise, or from any use or operation of any methods, products, instructions or ideas contained in the materials published in these presentations.\nBecause of the rapid advances in medical sciences, we recommend that independent verification of diagnoses and drugs dosages should be made.\nFurthermore, patients and the general public visiting the website should always seek professional medical advice.\nFinally, please note that ESO does not endorse any opinions expressed in the presentations.\nTo share your e-eso experience use:\n#e_ESO\n2', 'The image shows a slide from the European School of Oncology\'s 404th e-session. The session is titled "The impact of oncological surgery on the outcomes." The expert for the session is Prof. Charles M. Balch from the University of Texas MD Anderson Cancer Center, Houston, USA, and the discussant is Prof. Riccardo Audisio from Whiston Hospital, University of Liverpool, Prescot, UK. The slide also includes an extract from the e-ESO policy, which outlines the purpose of the website, the responsibility disclaimer, and a recommendation for independent verification of diagnoses and drug dosages. It also advises patients and the general public to seek professional medical advice when visiting the website and notes that ESO does not endorse any opinions expressed in the presentations. The slide ends with an invitation to share e-ESO experiences using the hashtag #e_ESO.', "The image shows a slide from an e-session, specifically e-session 404, which is titled 'The impact of oncological surgery on the outcomes'. The slide features the logos of e-ESO and e-ESO, indicating that the content is related to the e-ESO policy. The slide also includes the names and affiliations of the expert and discussant for the session, Prof. Charles M. Balch from the University of Texas MD Anderson Cancer Center, Houston, USA, and Prof. Riccardo Audisio from Whiston Hospital, University of Liverpool, Prescot, UK. The slide contains an extract from the e-ESO policy, which outlines the purpose of the website, the ownership of the materials, the disclaimer of responsibility for any injury or damage, the recommendation for independent verification of diagnoses and drug dosages, and the advice for patients and the general public to seek professional medical advice. It also notes that e-ESO does not endorse any opinions expressed in the presentations and provides a hashtag for sharing e-ESO experiences. The slide is numbered as slide 2.", 'The image shows a slide from an e-session, specifically e-session 404, which is about the impact of oncological surgery on the outcomes. The slide features the logos of e-ESO and e-ES, indicating that the session is accredited by these organizations. The session is presented by an expert, Prof. Charles M. Balch from the University of Texas MD Anderson Cancer Center, and a discussant, Prof. Riccardo Audisio from Whiston Hospital, University of Liverpool, Prescot, UK. The slide also includes an extract from the e-ESO policy, which outlines the purpose of the website, the ownership of the materials, the liability disclaimer, the recommendation for independent verification of medical information, and a reminder that e-ESO does not endorse any opinions expressed in the presentations. Additionally, there is a hashtag provided for sharing e-ESO experiences. The slide is numbered as slide 2.', 'At the end of the presentation, by closing the webcast window, you will be directed to the CME evaluation and multiple choice test.', 'Mastermind competidores serán alentados a comenzar la prueba después de haber completado la evaluación.', 'Como recordatorio, durante la sesión en vivo, todos los participantes pueden hacer preguntas en vivo en cualquier momento, simplemente presionando en el botón correspondiente en la parte superior de la página.', 'Aquí hay un extracto de la política que se publica en nuestro sitio web en su totalidad.', 'Questions will be received by Professor Ricardo Audisio, of the Whiston Hospital, University of Liverpool, Prescot, United Kingdom, who will discuss them with Professor Charles M. Balch, of the University of Texas MD Anderson Cancer Center, Houston, Texas, USA.', 'The slide is titled "e-session 404" and discusses the impact of oncological surgery on the outcomes. It features an expert, Prof. Charles M. Balch from the University of Texas MD Anderson Cancer Center, Houston, USA, and a discussant, Prof. Riccardo Audisio from Whiston Hospital, University of Liverpool, Prescot, UK. The slide includes an extract from the e-ESO policy, which is a website containing presentations aimed at providing new knowledge and competences, mainly designed for oncology professionals and other physicians interested in oncology. The policy states that the website\'s materials remain the property of the authors or ESO and that ESO is not responsible for any injury or damage caused by the use of the materials. It also advises independent verification of diagnoses and drug dosages and recommends seeking professional medical advice. The slide concludes with a note that ESO does not endorse any opinions expressed in the presentations and provides a hashtag, #e_ESO, for sharing e-ESO experiences.', 'Profesor, su línea ahora está abierta.\ne-sesion 404\nThe impact of oncological surgery on the outcomes\nExpert: Prof. Charles M. Balch, University of Texas MD Anderson Cancer Center, Houston, USA\nDiscussant: Prof. Riccardo Audisio, Whiston Hospital, University of Liverpool, Prescot, UK\nExtract from the e-ESO policy\nThe website contains presentations aimed at providing new knowledge and competences, and is intended as an informational and educational tool mainly designed for oncology professionals and other physicians interested in oncology.\nThese materials remain property of the authors or ESO respectively.\nESO is not responsible for any injury and/or damage to persons or property as a matter of a products liability, negligence or otherwise, or from any use or operation of any methods, products, instructions or ideas contained in the materials published in these presentations.\nBecause of the rapid advances in medical sciences, we recommend that independent verification of diagnoses and drugs dosages should be made.\nFurthermore, patients and the general public visiting the website should always seek professional medical advice.\nFinally, please note that ESO does not endorse any opinions expressed in the presentations.\nTo share your e-eso experience use:\n#e_ESO\n2', 'El impacto de la cirugía oncológica en los resultados', 'Esta es Charles Balch de Houston, Texas, y estoy encantado de unirme a este seminario para hablar sobre el impacto de los resultados de la cirugía oncológica. También estoy involucrado en este es mi buen amigo, el profesor Ricardo Adicesio, el ex presidente de la Sociedad de Cirugía Oncológica y el presidente electo de la Sociedad Europea de Cirugía Oncológica, y él conducirá un período de preguntas y respuestas al final de mi presentación.', 'El audio se traduce al español como sigue:\n\nCharles M. Balch, MD, FACS\nProfesor de Cirugía\nUNIVERSIDAD DE TEXAS MD ANDERSON CANCER CENTER HOUSTON, TEXAS\nEditor-in-Chief, Annals of Surgical Oncology\nPresidente, Cancer Expert Now Internacional, Inc.\nPast-Presidente, Sociedad de Cirugía Oncológica\nPast Ejecutivo VP y Chief Executive Officer, ASCO', 'The image shows a slide from a presentation titled "Global Surgery 2030: evidence and solutions for achieving health, welfare, and economic development." The slide includes an executive summary and a quote emphasizing the importance of access to surgical care. The executive summary discusses the progress made in global health over the past 25 years, the challenges faced by low- and middle-income countries (LMICs) in accessing safe, essential surgical and anesthesia care, and the need for a national surgical plan. It highlights that 5 billion people lack access to safe, affordable surgical and anesthesia care, and that 143 million additional surgical procedures are needed in LMICs each year to save lives and prevent disability. The quote on the slide states that "Access to surgical care is essential for reduction of mortality and morbidity from surgical conditions... More than 2 billion people have no access to surgery and anesthesia. This figure is probably an under estimate... Universal access to safe, affordable surgical care is essential..." The slide also includes the names of the authors and a reference to the Lancet Commission on Global Surgery 2015. The slide is part of a larger presentation, as indicated by the slide number 4 at the bottom.', 'The image shows a slide from a presentation titled "Global Surgery 2030: evidence and solutions for achieving health, welfare, and economic development." The slide is part of a report by The Lancet Commissions and includes an executive summary and a list of key messages. The executive summary discusses the progress made in global health over the past 25 years, the uneven progress, and the need for safe, essential surgical and anesthesia care in low- and middle-income countries (LMICs). It highlights that 5 billion people do not have access to safe, affordable surgical and anesthesia care when needed, with the worst access in low-income and lower-middle-income countries. The slide also mentions that 143 million additional surgical procedures are needed in LMICs each year to save lives and prevent disability, and that 313 million procedures are undertaken worldwide each year. The key messages presented include the need for universal access to safe, affordable surgical care. The slide also includes a quote emphasizing the importance of access to surgical care for reducing mortality and morbidity from surgical conditions and suggests that the figure of more than 2 billion people without access to surgery and anesthesia is probably an underestimate. The slide is marked as page 4 of the report.', 'The image shows a slide from a presentation titled "Global Surgery 2030: evidence and solutions for achieving health, welfare, and economic development" by The Lancet Commissions. The slide includes an executive summary discussing the importance of access to safe, affordable surgical care for reducing mortality and morbidity from surgical conditions. It highlights that 5 billion people lack access to such care, with the situation being worst in low-income and lower-middle-income countries. The slide also mentions that 143 million additional surgical procedures are needed annually in low- and middle-income countries to save lives and prevent disability. The slide is part of a report published online on April 27, 2015, and is the 386th issue of The Lancet. The slide is marked as page 507 and includes a note that Prof. Rodas died on March 2, 2015, and the report is dedicated to him. The slide also features logos of CrossMark and CrossRef.', 'En un artículo subsecuente en el que Dr. Odiccio y yo estuvimos involucrados, en Lancet Oncology, nos enfocamos en la cirugía de cáncer a nivel global y cómo entregar cirugía segura, asequible y a tiempo. Y yo realmente recomendaría este artículo para todos los que estén leyendo.', 'El resumen del artículo publicado en Lancet Oncología en 2015 y, como puedes ver de las flechas, Dr. Odiccio y yo fuimos coautores de este. Y parte del resumen, que haremos resaltar brevemente, indicó que para el año 2013, que es solo 13 años de ahora, estimamos que 45 millones de procedimientos quirúrgicos serán necesarios en todo el mundo. Y aún menos del 25% de los pacientes con cáncer en todo el mundo realmente obtienen cirugía segura, asequible y o a tiempo.', 'El audio se traduce como sigue:\n\n"El Lancet Oncology"', 'El audio se traduce como sigue:\n\n"Y ofrecimos soluciones que incluyen: un sistema público regulado mejor, especialmente en la capacitación y certificación de cirujanos, asociaciones internacionales, centralización de servicios quirúrgicos, y volveré a eso más tarde, nuevos ensayos clínicos de cirugía y nuevas aproximaciones para mejorar la calidad y escalar el sistema quirúrgico de cáncer a través de la educación y capacitación."\n\nLa traducción al español es:\n\n"Y ofrecimos soluciones que incluyen: un sistema público regulado mejor, especialmente en la capacitación y certificación de cirujanos, asociaciones internacionales, centralización de servicios quirúrgicos, y volveré a eso más tarde, nuevos ensayos clínicos de cirugía y nuevas aproximaciones para mejorar la calidad y escalar el sistema quirúrgico de cáncer a través de la educación y capacitación."', 'El audio se traduce como sigue:\n\nLa Comisión Oncología del Lancet\n\n- "Los problemas de acceso a la cirugía, una falta de inversión en sistemas de cirugía pública, la inversión baja en investigación y formación, y los vacíos educativos son ampliamente extendidos"\n- "Las soluciones incluyen: sistemas de cirugía pública mejor regulados, asociaciones internacionales, centralización de servicios quirúrgicos de alto nivel, ensayos clínicos quirúrgicos innovadores y nuevas estrategias para mejorar la calidad y la escalabilidad de los sistemas quirúrgicos de cáncer a través de la educación y la formación"\n\n7', 'The image shows a slide titled "THE LANCET ONCOLOGY COMMISSION" with two bullet points of text. The first bullet point states that achieving good outcomes in modern cancer surgery is technically demanding and that outcomes improve when such surgery is undertaken by specialized teams in high volume centers. The second bullet point discusses the access to high-quality training in these units, with suitable accreditation and quality control for aspiring oncologic surgeons, which has yet to be formally met by regulatory authorities in a number of countries. The slide number is indicated as 8. The text is in English and there is no audio content to translate.', 'The image shows a slide titled "THE LANCET ONCOLOGY COMMISSION" with two bullet points of text. The first bullet point discusses the technical demands of achieving good outcomes in modern cancer surgery and acknowledges that outcomes improve when surgery is performed by specialized teams in high-volume centers. The second bullet point emphasizes the importance of access to high-quality training in these units, with suitable accreditation and quality control for aspiring oncologic surgeons, which has not yet been formally met by regulatory authorities in many countries. The slide number is indicated as 8 at the bottom right corner. The text is in English and there is no audio content to translate.', 'The image shows a slide titled "THE LANCET ONCOLOGY COMMISSION" with two bullet points of text. The first bullet point discusses the technical demands of achieving good outcomes in modern cancer surgery and acknowledges that outcomes improve when surgery is performed by specialized teams in high-volume centers. The second bullet point emphasizes the need for access to high-quality training in these units, with suitable accreditation and quality control for aspiring oncologic surgeons, which has not yet been formally met by regulatory authorities in many countries. The slide number is indicated as 8 at the bottom right corner. The text is clear and legible, and there is no need for a caption as the content is straightforward and directly related to the topic of oncology training and surgery outcomes.', 'El audio se traduce como sigue:\n\n"Y aunque los países de alto ingreso, como muchos países de Europa y de los Estados Unidos, están impulsando la mayor especialización, globalmente, hay una necesidad de cirujanos con una formación mucho más amplia. Los sociedades profesionales de cirugía deben desempeñar un papel más destacado en este área. Y finalmente, que los países de alto ingreso y medio ingreso tengan un deber y una oportunidad para expandir sus ofertas educativas en cirugía de cáncer a los países de bajo ingreso a través de intercambios bilaterales, un mayor uso de la tecnología mejorada para el aprendizaje, y asociaciones para desarrollar contenido específico de currículo sobre cáncer en programas de residencia de cirugía general."\n\nLa traducción se realiza manteniendo el significado y el contexto del texto original.', 'Estas son algunas ejemplos de las alianzas y la disponibilidad de los programas globales en educación, incluyendo la Sociedad de Oncología Quirúrgica, la Sociedad Europea, la Asociación Británica y varias otras organizaciones, como ESMO, ASCO y un grupo que estoy involucrado en llamado Salud Voluntarios Internacionales que está basada en los Estados Unidos.', 'Ahora, en las próximas dos diapositivas, quiero enfatizar la importancia y el valor en términos de calidad de atención en el especializamiento.', 'El audio se traduce como sigue:\n\n"¿Por qué los cirujanos necesitan especializarse o subspecializarse? \n- Avances rápidos y sustanciales en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la investigación del cáncer \n- Avances en la', 'El audio se traduce como sigue:\n\n"Y esto ha llevado a nuevas combinaciones y secuencias de tratamientos de cáncer, incluyendo terapias neoadjuvante y adjuvante en los pacientes quirúrgicos."', 'El audio se traduce como sigue:\n\n"¿Por qué los cirujés necesitan especializarse o subspecializarse? \n- Avances rápidos y sustanciales en la investigación del cáncer\n-- Diagnósticos y biomarcadores nuevos\n-- Nuevos medicamentos, hormonas, terapias dirigidas\n-- Nuevas combinaciones y secuencias de tratamiento del cáncer, incluyendo terapias neoadjuvantes\n- Avances importantes en tecnologías quirúrgicas en el quirófano\n-- Cómo podemos excisear los cánceres con mejores y más seguros resultados\n-- Importancia de la rehabilitación y restauración de la función\n- Cambios en la entrega de atención al cáncer a través de "Centros de Cáncer" y equipos multidisciplinarios de especialistas en oncología\n- Evidencia de que los "Centros de Cáncer de alto volumen" y especialistas quirúrgicos tienen mejores resultados para tratar los cánceres complejos."', 'The image is a slide from a presentation with a dark blue background and yellow text. The title of the slide is "WHY DO SURGEONS NEED TO SPECIALIZE... OR SUBSPECIALIZE?" Below the title, there are several bullet points that discuss the reasons why surgeons should consider specializing or subspecializing. The bullet points mention rapid and substantial advances in cancer research, major advances in surgical technologies in the operating room, changes in the delivery of cancer care through cancer centers and multidisciplinary teams of oncology specialists, and evidence that high-volume cancer centers and surgical specialists have better outcomes for treating complex cancers. The slide number is indicated at the bottom right corner as "11."', 'El audio se traduce como sigue:\n\n"Y hay muchos cambios en la entrega de atención médica para el cáncer a través del concepto de centros de cáncer y equipos multidisciplinarios de especialistas en oncología, que incluyen cirujanos que eligen pasar parte de su tiempo como parte de un equipo multidisciplinario y participando en planes de tratamiento con nuestros colegas en medicina, radiología, oncología, patología y radiología."\n\nLa traducción al español es:\n\n"Y hay muchos cambios en la entrega de atención médica para el cáncer a través del concepto de centros de cáncer y equipos multidisciplinarios de especialistas en oncología, que incluyen cirujanos que eligen pasar parte de su tiempo como parte de un equipo multidisciplinario y participando en planes de tratamiento con nuestros colegas en medicina, radiología, oncología, patología y radiología."', 'El audio se traduce como sigue:\n\n"Y si te mostraría solo unas pocas diapositivas de evidencia que que el centro de cáncer de alto volumen y los especialistas en cirugía tienen mejores resultados para tratar el cáncer complejo."\n\nThe slide is titled "WHY DO SURGEONS NEED TO SPECIALIZE... OR SUBSPECIALIZE?" and lists several bullet points. The bullet points discuss the rapid and substantial advances in cancer research, major advances in surgical technologies in the operating room, changes in the delivery of cancer care through "Cancer Centers" and multidisciplinary teams of oncology specialists, and evidence that "high volume" cancer centers and surgical specialists have better outcomes for treating complex cancers. The slide number is 11.', 'Y\n', 'El audio muestra un ejemplo de una enfermedad común, que es una publicación en el "British Journal of Cancer" de 2004. En ella, se cita que "Los pacientes con cáncer de mama tratados por especialistas en comparación con los no especialistas, tenían un riesgo de tratamiento inadecuado del pecho (24% vs 47%), un riesgo cinco veces menor de inadecuado estaging axilar (8% vs 40%) y un riesgo nueve veces menor de inadecuado tratamiento axilar (4% vs 38%)." Además, se menciona que "Las tasas de recurrencia local fueron 57% más bajas (13% vs 23% a los ocho años) y el riesgo de muerte por cáncer de mama fue 20% más bajo para las mujeres tratadas en unidades de especialistas, después de permitir por caso-mix y adjuvant therapies." Finalmente, se concluye que "El manejo quirúrgico adecuado del cáncer de mama es fundamental para mejorar el resultado del cáncer de mama, independientemente de dónde se delivered."', 'The image is a slide from a presentation discussing the importance of specialist treatment for breast cancer and its impact on survival rates. The slide is titled "Why does specialist treatment of breast cancer improve survival? The role of surgical management." and cites a study by Kingsmore D, et al published in the Journal of Breast Cancer in 2004. The study found that breast cancer patients treated in specialist units had a significantly lower risk of inadequate treatment of the breast, inadequate axillary staging, and inadequate axillary treatment compared to those treated in nonspecialist units. The slide also mentions that local recurrence rates were 57% lower and the risk of death from breast cancer was 20% lower for women treated in specialist units. The conclusion drawn from the study is that adequate surgical management of breast cancer is fundamental to improving the outcome from breast cancer, regardless of where it is delivered. The slide is numbered 12, indicating its position within the presentation.', 'The image shows a slide from a presentation discussing the impact of specialist treatment on breast cancer survival rates. The slide is titled "Why does specialist treatment of breast cancer improve survival? The role of surgical management." and cites a study by Kingsmore D, et al published in the British Journal of Cancer in 2004. The study found that breast cancer patients treated in specialist units had significantly lower risks of inadequate treatment, local recurrence, and death from breast cancer compared to those treated in nonspecialist units. The slide emphasizes that adequate surgical management of breast cancer is fundamental to improving outcomes, regardless of where it is delivered. The slide is numbered 12, indicating its position within the presentation.', 'El texto en la imagen está en inglés.', 'Aquí hay algunos ejemplos seleccionados de la revista New England Journal of Medicine que muestran la relación entre el volumen hospitalario y la mortalidad quirúrgica en los Estados Unidos. El punto clave aquí es que, para la colestectomía, no hubo diferencia en la mortalidad ajustada comparando el volumen hospitalario y la mortalidad ajustada. Sin embargo, hubo una diferencia significativa en la mortalidad relacionada con la cirugía para la gastrrectomía, la esofagectomía y la resección pancreática.', 'Continuing in another paper, we are looking at surgeon volume. The last paper was on hospital volume. There were some operations, such as we are looking at adjusted operative mortality. There was not much difference in resection for lung cancer or cystectomy, but there was a highly significant difference comparing surgeon volume and operative mortality for esophagectomy and pancreatic resections.', 'Ahora, este es un interludio para recordarles que sus opiniones son importantes. Pueden hacer preguntas y enviar comentarios en cualquier momento. Así, al final de esta presentación, profesor Adisio y yo podemos tener una conversación con sus preguntas.', 'Volumen no es el factor más importante', 'Este es un ejemplo de cómo la presentación del paciente y la comorbididad, especialmente entre los pacientes más antiguos, también puede afectar el resultado.', 'Así, en resumen de algunos de estos artículos del "New England Journal of Medicine" y de otras publicaciones, si se compara la mortalidad postoperatoria en centros especializados con hospitales comunitarios en los Estados Unidos, no hubo diferencia para una variedad de tumores que involucran el pecho, melanoma, ovario, prostate, tiroido y útero.', 'Sin embargo, hubo diferencias entre los pacientes de alto riesgo comparando los centros especializados con los hospitales comunitarios, mostrando que para el riesgo más alto y alto riesgo, hubo una diferencia significativa en especializaciones, mientras que en pacientes de bajo riesgo, no había diferencia entre los centros especializados y los hospitales comunitarios.', 'Y', 'Aquí hay solo una otra paper de Dr. Birkmeier en el New England Journal of Medicine, nuevamente mostrando, basándose en el volumen de cirugía y el volumen de hospital, mostrando diferencias significativas de volumen de volumen y resultado con respecto a la mortalidad de 30 días relacionada con la cirugía.', "The image is a slide from a presentation by the American College of Surgeons, discussing the National Surgical Quality Improvement Program (NSQIP). The slide highlights the program's impact on hospitals across the United States, with 66% of hospitals improving mortality rates and 82% improving morbidity rates. It also notes that even the worst-performing hospitals were more likely to improve, and that good performers also saw improvements. The slide mentions that the number of low outliers (good) increased, while the number of high outliers (bad) decreased. Additionally, it states that hospitals avoided an estimated 250 complications per year. The slide is part of a larger presentation, as indicated by the footer, and is numbered 23 in the sequence.", 'El audio se traduce como sigue:\n\n"Y que el número de bajos outliers aumentó y el número de altos outliers disminuyó, lo cual es bueno basado en estos programas de calidad iniciados a nivel nacional."', 'El programa resultó en que cada hospital evitó un promedio de 250 complicaciones por año.', 'Así como está, quiero que envíes tus preguntas enviando comentarios en cualquier momento.', 'Y aquí hay un punto muy importante: \nAunque hablamos de cirugía oncológica y la especialización en la misma, la Junta Americana de Cirugía, que certifica a todos los cirujanos generales y a las especialidades de cirugía general, incluye la cirugía oncológica, incluyendo la cirugía de cabeza y cuello, entre sus áreas centrales que se incluyen en sus exámenes de certificación.', 'El audio trata sobre la importancia de la cirugía en el manejo del cáncer, así como su papel en la gestión de otras condiciones como lesiones, cirugía vascular pediátrica, cuidados críticos y otras criterios.', 'Y', 'El audio se traduce como sigue:\n\n"Y nuestro currículo, tanto en los Estados Unidos como en estos currículums globales, incluye las características distintivas de los oncólogos quirúrgicos que manejan operaciones complejas, que son parte de un equipo multidisciplinario que incluye a los médicos y oncólogos en el tratamiento de los pacientes en el inicio de su diagnóstico y participan como oncólogos en la gestión a largo plazo del paciente de cáncer."', 'In', 'Ahora, en un nivel formal, y esto se relaciona con las certificaciones en oncología médica, pediátrica y radiación, ahora hay una entidad separada en oncología general compleja.', 'El audio se traduce como sigue:\n\n"Y este programa de entrenamiento de dos años implica la capacitación adicional en el cuidado multidisciplinario del cáncer, la gestión quirúrgica de las condiciones oncologicas, el asesoramiento al paciente sobre la prevención y la intervención del cáncer, y, muy importante, la participación y liderazgo en la investigación clínica y el diseño de ensayos clínicos."\n\nY este programa de entrenamiento de dos años implica la capacitación adicional en el cuidado multidisciplinario del cáncer, la gestión quirúrgica de las condiciones oncologicas, el asesoramiento al paciente sobre la prevención y la intervención del cáncer, y, muy importante, la participación y liderazgo en la investigación clínica y el diseño de ensayos clínicos.', 'Y', 'Y y no, lo digo porque en mis viajes a otros países, todavía hay muchos centros comunitarios. Estamos haciendo un bioprobarte abierto para hacer la detección de cáncer de mama es el estándar de cuidado.', 'La imagen muestra un procedimiento quirúrgico donde un médico está realizando una biopsia guiada por ultrasonido. El procedimiento implica la inserción de un aguja en el tejido del paciente, guiada por el ultrasonido para localizar el área de interés. El médico usa una herramienta especial para extraer una muestra de tejido, que luego se examinará para determinar si hay algún problema de salud. Este procedimiento es comúnmente utilizado en el tratamiento de tumores y otras condiciones médicas.', 'Ahora, en estas próximas pocas diapositivas, quiero enfatizar lo que es un componente muy importante de los pacientes quirúrgicos aquí en el cáncer de mama, en pacientes que presentan con el estadio 3 de la enfermedad, que con la terapia contemporánea de quimioterapia y terapia sistemática y dirigida, los pacientes están ahora alcanzando un resultado patológico completo del 40 a 60% de tales pacientes, y esto va a cambiar la forma en que se gestionan los pacientes quirúrgicos, porque la cirugía puede no ser la primera modalidad de tratamiento.', 'Aquí en el cáncer de mama en los Estados Unidos, la mayoría de los pacientes con cáncer de mama tienen el estadio 2 y 3. Y creo que en muchos centros en Europa, especialmente para el estadio 3, la terapia sistémica es la primera terapia y luego la cirugía es la segunda terapia. Entonces tenemos que tener muchas discusiones con nuestros colegas en medicina oncológica sobre el momento de la cirugía, el papel del nodo sentral, que es diferente en este caso, pero esto representa el futuro de nuestros tratamientos para muchas enfermedades para las que el cáncer de mama es el prototipo.', 'Aquí hay otro ejemplo. Aquí está un paciente con melanoma avanzado en el estómago y el pecho, que fue tratado con terapia inmunitaria. Después de la terapia inmunitaria, tuvo una reducción tan significativa que facilitó la operación. En este paciente, cuando se eliminaron todos estos masas, contenían inflamación crónica y cicatrización. Este paciente tuvo una respuesta patológica completa, que no habría sido posible sin la resección quirúrgica de las masas de tumor que inicialmente se detectaron.', 'El futuro del tratamiento sistémico de cáncer está cambiando drásticamente. Hay más tratamientos efectivos y muchos de los nuevos tratamientos serán terapias orales que serán menos tóxicas y tendrán un impacto significativo en los pacientes quirúrgicos.', 'Aquí hay un ejemplo de dónde están disponibles nuevos agentes orales o subcutáneos para una variedad de tipos de cáncer.', 'Aquí hay algunos ejemplos de nuevos agentes de cáncer oral o subcutáneos:\n\nAnastrozole\nExemestane\nLetrozole\nCapecitabina\nErtoline\nImatinib\nGefitinib\n\nPegalated interferon\nVematerfinib\nTrametininb\nLapatinib\nSunitinib\nSorafenib\nDenosumab\n\nTHERE ARE OVER 1000 NEW CANCER DRUGS IN VARIOUS STAGES OF CLINICAL DEVELOPMENT!!', 'Esta es una paciente que tuvo un melanoma metastático extenso, que incluso después de dos semanas de terapia, se podía ver en el PET-CT que hubo una reducción dramática en este melanoma extenso.', 'Y\nEs\nde\nlos\nagentes\nson\ntan\npotentes\nque\nya\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\nbegin\n', 'El avance más importante que está sucediendo es el uso de la inmunoterapia con los nuevos inhibidores de punto de control, que ahora tienen actividad en más de 17 tipos diferentes de cáncer. Y esto representa nuestro futuro, que se utilizará cada vez más en pacientes quirúrgicos.', 'Claro, aquí tienes un ejemplo de una respuesta positiva a la terapia inmunoterapéutica en un caso de metástasis de melanoma grande en el pecho y el hombro. Después de recibir una sola dosis de terapia inmunoterapéutica, este paciente mostró una respuesta patológica completa, demostrada por la cirugía de excisión de los tejidos restantes.', 'El presidente de los Estados Unidos, Jimmy Carter, se enfrentó a un cáncer metastático de melanoma. A la edad de 92 años, fue tratado con una sola dosis de un medicamento contra el PD-1, una nueva terapia inmunitaria.', 'El presidente de Estados Unidos, Jimmy Carter, ha sido tratado con un nuevo medicamento contra la melanoma metastásica. Ha estado en remisión durante más de 14 meses. El medicamento, que se llama pembrolizumab, ha demostrado ser eficaz en el tratamiento de esta forma de cáncer. Jimmy Carter, quien tiene 92 años, ha sido tratado con este medicamento y ha logrado una remisión de 16 meses. El medicamento ha sido elogiado por su eficacia y ha sido considerado uno de los medicamentos más prometedores en el tratamiento de cáncer. Jimmy Carter ha expresado su gratitud por la ayuda que ha recibido y ha compartido su experiencia con el público.', 'Quería mostrar otro ejemplo de un paciente con cáncer de cabeza y cuello que tenía 96, enfermedad avanzada y, incluso después de 4 semanas con este tratamiento anti-PD-1 único, tuvo una respuesta dramática.', 'Así como si estuvieras viendo estos tipos de reacciones en pacientes con un estado de salud más avanzado, puedes estar seguro de que esto se trasladará a la atención de pacientes con un estado de salud más temprano, como el estado 2 y el estado 3 de cáncer.', 'Continuing Education For Practicing Surgeons is Essential to Keep Up With Advances in Surgery and in Oncology\nLive meetings\nAcademic Affiliations\nJournals\nLive Webinars\nEmail Collaboration\n42', 'Y aquí, de nuevo, hay una variedad de formas en que esto está sucediendo, especialmente con las reuniones en vivo, especialmente las que se realizan por la Sociedad de Oncología (SSO) y la Sociedad Europea de Oncología (ESO) y la Asociación de Oncólogos (ASCO), pero también a través de asociaciones con centros terciarios, como en mi institución, con hospitales comunitarios, con avances en revistas, con colaboraciones por correo electrónico o con seminarios en vivo, como este seminario que se está llevando a cabo por la Sociedad Europea de Oncología.', 'Both Dr. Adiccio and I are the editors-in-chief of our two respective journals, and we would recommend those to you for keeping up with the field, both the online version and the print version.', 'El Sociedad de Oncología también tiene ahora un programa de autoevaluación que se realiza para ayudar a los cirujanos a mantenerse al día con el campo de la oncología.', 'Y', 'El telemedicina está siendo utilizada cada vez más no solo en los Estados Unidos, sino también en todo el mundo, tanto para la educación médica a los médicos, especialmente a los que trabajan en el entorno comunitario y en áreas rurales, como también para los servicios consultivos especializados, especialmente en ayudar en la toma de decisiones en el cuidado del paciente en otras ubicaciones o incluso en otros países.', 'El audio se traduce como sigue:', 'El audio se traduce como: "I\'ve even been involved in a company called Cancer Expert Now, and I\'m the president of Cancer Expert Now International, where we do consultation with physicians."', 'Esta imagen muestra un ejemplo de una de nuestras neurooncologistas realizando una consulta con un médico en otro país sobre un paciente con un tumor cerebral complicado.', 'Esta imagen muestra otro ejemplo de cómo fui en Hangzhou, China, y realizamos un panel virtual de telemedicina aquí con el Dr. Hope Rugo, quien estaba en San Francisco en ese momento, discutiendo a un paciente complejo con cáncer de mama', 'Aquí está, justo después de esto, y un video de televideo en el mismo lugar sobre un paciente de cáncer colorrectal complejo, y Dr. Glenn Bolch, quien es el jefe de cirugía colorrectal en Emory en Atlanta.', 'The image shows a presentation slide titled "The Virtual Tumor Board #ASCO2016 - Oral Presentation." The slide contains bullet points highlighting the benefits of the Virtual Tumor Board, with statistics indicating that 91% of attendees found it helpful in managing their patients, 100% felt that the quality of care was improved, and 100% indicated improved confidence through participation. The presenter is listed as Minggui Pan MD, PhD, and the session is part of the Science of Quality Poster Session B. The slide number is 49.\n\nThe slide also includes a section of a poster with the title "Improving Care Quality with Soft Tissue and Bone Sarcoma by Establishing a National Virtual Tumor Board and Electronic Consultation Platform." The poster is divided into three sections: Background, Methods, and Results. The Results section includes a pie chart and a bar graph, but the specific data is not clear from the image. The poster appears to be related to the topic of the presentation and provides additional information on the effectiveness of the Virtual Tumor Board.', 'El audio se traduce al español como sigue:\n\nTítulo: La Junta Virtual del Tumor #ASCO2016 - Presentación Oral\n\n- 91% de los asistentes a la Junta Virtual del Tumor encontraron que era muy útil en la gestión de sus pacientes\n- 100% de los asistentes sentían que la calidad del cuidado mejoró\n- 100% de los participantes indicaron que su confianza mejoró a través de la participación\n\nPresentación: Minggui Pan MD, PhD\nSesión de Posters: B: Ciencia de la Calidad\n\nPágina 49', "All right, and again, we're almost finished our talk, and I'm going to do a summary shortly.\n\nThe Virtual Tumor Board #ASCO2016 - Oral Presentation\n\n• 91% of Virtual Tumor Board attendees found it to be very helpful in managing their patients\n• 100% of attendees felt that quality of care was improved\n• 100% of participants indicated improved confidence through participation\n\nPresentation: Minggui Pan MD, PhD\nPoster Session B: Science of Quality\n49", 'Y', 'El audio se traduce al español como sigue:\n\n"ADOPTING TO CHANGE IN A CANCER PRACTICE:\n\n• Almost all cancer patients in the United States will receive more than one modality of treatment; Contemporary cancer care is a team approach!\n• Cancer is becoming a chronic or curable disease\n– Survivorship; second primaries; treatment compliance\n• The oncology field is one of the most rapidly advancing and changing specialty in all of medicine\n– Important to validate advances through clinical trials\n• Importance of physician continuing education for a lifetime (through SSO, ESSO, ASCO, etc)', 'El audio habla sobre la adopción de cambios en un pr', 'El audio se traduce como sigue:\n\n"ADOPTING TO CHANGE IN A CANCER PRACTICE:\n\n- Almost all cancer patients in the United States will receive more than one modality of treatment; Contemporary cancer care is a team approach!\n- Cancer is becoming a chronic or curable disease\n- Survivorship; second primaries; treatment compliance\n- The oncology field is one of the most rapidly advancing and changing specialty in all of medicine\n- Important to validate advances through clinical trials\n- Importance of physician continuing education for a lifetime (through SSO, ESSO, ASCO, etc)', 'El audio se traduce como sigue:', 'El audio se traduce como sigue:\n\n"Y después de todos estos tratamientos intensivos, los supervivientes en pacientes que están viviendo por años después de su tratamiento están en riesgo de no cumplir con las indicaciones de tratamiento y las consecuencias de su tratamiento intensivo y multidisciplinario."', 'El campo de la oncología es uno de los campos de medicina que está cambiando y avanzando más rápidamente. Esto significa que los cirujés deben mantenerse al día a través de la educación continua y incorporar estos cambios en el currículo de sus estudiantes.', 'El audio se traduce como sigue:\n\n"ADOPTING TO CHANGE IN A CANCER PRACTICE:\n\n• Almost all cancer patients in the United States will receive more than one modality of treatment; Contemporary cancer care is a team approach!\n• Cancer is becoming a chronic or curable disease\n– Survivorship; second primaries; treatment compliance\n• The oncology field is one of the most rapidly advancing and changing specialty in all of medicine\n– Important to validate advances through clinical trials\n• Importance of physician continuing education for a lifetime (through SSO, ESSO, ASCO, etc)', 'El audio habla sobre la importancia de la educación continua para los profesionales, especialmente en el campo de la oncología, para mejorar el tratamiento y la supervivencia de los pacientes con cáncer.', 'Surgeons, in order to achieve a better outcome, are going to need to partition their patients and identify those for which specialization or referral to high-volume centers specializing in certain diseases will have a better outcome for treating patients with complex or advanced cancers.', 'SURGEONS NEED TO SPECIALIZE TO ACHIEVE BETTER OUTCOME!\n\n• Higher volume surgeons achieve better outcomes in treating patients with complex or advanced cancers\n• There is a patient and public demand for getting optimal cancer care from specialists and multidisciplinary cancer centers\n• BUT...this does NOT mean that all cancer patients need to be treated by specialists, especially if there diagnosis is made early and their treatment is simple and does not need multiple specialties for their care\n• Surgical oncologists should be educational and research leaders among the general surgery community\n\n52', 'SURGEONS NEED TO SPECIALIZE TO ACHIEVE BETTER OUTCOME!', 'El audio habla sobre la importancia de que los pacientes permanezcan en su entorno local lo máximo posible y cómo la telemedicina puede ayudar en este aspecto. La telemedicina puede proporcionar a los pacientes, incluso en hospitales más pequeños, la oportunidad de recibir asesoramiento y planificación de tratamiento con la ayuda de especialistas a través de telemedicina o consultas.', 'SURGEONS NEED TO SPECIALIZE TO ACHIEVE BETTER OUTCOME!\n• Higher volume surgeons achieve better outcomes in\ntreating patients with complex or advanced cancers\n• There is a patient and public demand for getting optimal cancer care from specialists and multidisciplinary cancer centers\n• BUT...this does NOT mean that all cancer patients need to be treated by specialists, especially if there diagnosis is made early and their treatment is simple and does not need multiple specialties for their care\n• Surgical oncologists should be educational and research\nleaders among the general surgery community\n52', 'Conclusions:\n• Profesionales sociedades, tales como la Sociedad de Oncología Quirúrgica, el European SSO, el American College of Surgeons, la American Society of Clinical Oncology son componentes importantes de definir los estándares para tratar a pacientes quirúrgicos con cáncer y proporcionar educación/training/workshop para lograr el cuidado de cáncer óptimo.\n• Las alianzas entre nuestras sociedades profesionales y nuestras instituciones médicas acelerarán nuestro progreso, avanzaremos nuestra especialidad y mejoraremos el cuidado de nuestros pacientes.', 'Traducción al español:', 'Así es mi conclusión. Ahora la pasaré a profesor Adesio para sus comentarios y para responder a cualquier pregunta que pueda haber hecho durante esta discusión.', 'CONCLUSIONES:\n• Los sociedades profesionales, como la Sociedad de Oncología Quirúrgica, la European SSO, la American College of Surgeons, la American Society of Clinical Oncology son componentes importantes para definir los estándares para tratar a pacientes quirúrgicos con cáncer y proporcionar educación/training/workshop para lograr el cuidado óptimo de cáncer.\n• Las alianzas entre nuestras sociedades profesionales y nuestras instituciones médicas acelerarán nuestro progreso, avanzaremos nuestra especialidad y mejoraremos el cuidado de nuestros pacientes.', 'Gracias, Charles. No tenemos palabras para agradecer a Profesor Balscher por resumir la presente y el futuro de la buena calidad de la cirugía de cáncer a nivel mundial.', 'Y', 'Así es, por favor, ¡aquí está!', "That's absolutely true, and I think that the professional organizations and governments in each nation need to insist that their citizens get high-quality care and ensure that there are well-trained surgeons, that there are organizational systems such as regional cancer centers, and I think increasingly that there are relationships internationally with individuals or with organizations, medical centers and cancer centers that can be of assistance for consultation and complex patients, and in the education of the surgeons in the local communities, and even in the more provincial or national cancer centers and hospitals.", 'Para compartir tu experiencia e-ESO, usa #e_ESO.', 'Sí, creo que este es un tema muy caliente y tal vez deseemos pasar un poco más de tiempo en cómo es mejor facilitar esto.', 'We have just pointed out how the network is there, and I would encourage young surgeons in training to feel absolutely free to follow their dreams, study their topics, and when they meet the big professor at the meeting, pull the slide and contact the specialist, the top specialist, and feel free to communicate with them, perhaps engage in the discussion and feel comfortable enough to take advantage of the large network that is available through the American Society of Surgical Oncology and the European one as well. We are currently working on facilitating this for developing countries in Africa through the unrestricted grant that pharmaceutical companies are making available. This is, in my mind, the way forward.', 'Así es como se hace: para compartir tu experiencia e-ESO, usa #e_ESO.', 'Creo que el punto clave es que no podemos mandar estas modificaciones. La iniciativa debe venir de los cirujés en cada comunidad, de sus hospitales y de sus gobiernos, tanto sus gobiernos locales como sus gobiernos nacionales, para aumentar el nivel de expectativas para un cuidado de alta calidad y oportuno.', 'E-sesion\nPregunta & Respuesta\nClick on the Q&A button\nto send your questions and comments\nTo share your e-eso experience use:\n#e_ESO', '¿Hemos de decir una palabra en advertencia? ¿Algo fácil de decir? ¿No quieres un escaneo PET cuando no tienes un escáner de ultrasonido?', 'You might consider not purchasing robotic equipment when you are missing basic laparoscopic stuff. And therefore, you need to develop the system, as you were saying, step by step according to the local facilities at the regional and provincial level.', 'Sí, es absolutamente cierto. He estado en un gran hospital en África, que tiene un muy caro radioterapia terapia máquina que está en cajas porque no tienen el fisicista de radioterapia o el técnico de radioterapia o el servicio, pero tienen la máquina en una caja que alguien compró, pero es más que eso.', 'La sesión de preguntas y respuestas en línea se basa en la planificación a nivel de gobierno y en el nivel de hospital para definir no solo los gastos de capital en edificios y equipos, sino también la entrega de los servicios por individuos calificados y entrenados en todos los niveles.', "So, Professor Bardach, we always have enemies and allies when we want to achieve a target. And that's a bit of a silly and restrictive way to look at it. But it has been the case that most surgical oncologists would blame the general surgeons for not embracing an oncological attitude or philosophy. And on the other hand, some medical oncologists are wiping surgeons' skills under the carpet.", 'Estamos en lo correcto al decir que realmente necesitamos a todos y que estamos bienvenidos a todos a bordo, porque esta es una discusión en curso que es solo un placer crear juntos como se desarrolla.', 'Me acuerdo y creo que el punto aquí es que el cirujano tiene que estar involucrado en el tratamiento.', 'Como ya hemos hablado, hay otras especialidades de cirujías que pasaron toda su vida en el quirófano y permitieron que la persona con formación en medicina tomara todas las decisiones. Y ahora, en esa especialidad, y estoy hablando de cirugía cardíaca, la cirujía es la última opción después de que todos los tratamientos médicos han fallado.', "And my concern is that if the surgeon does not bring the surgical perspective to the treatment planning, and is engaged with the medical oncologist and radiation oncologist, the patients may not get the full advantage of a true multidisciplinary treatment approach that involves surgery. But we are best trained and have the experience to determine what's the best timing of surgery in some combination and sequence.", 'E-sesion\nPregunta & Respuesta\nClick on the Q&A button\nto send your questions and comments\nTo share your e-eso experience use:\n#e_ESO', 'Así es como se hace:', 'Sí, es bueno. Así es como funciona. Primero, haz clic en el botón de Preguntas y Respuestas. Luego, escribe tus preguntas y comentarios. Finalmente, presiona el botón de enviar para compartir tus experiencias e-ESO.', 'Estoy aprovechando este tiempo porque entiendo de la organizadora que hay un problema con las preguntas que están llegando y que no se ha hecho disponible hoy, y que es un problema técnico.', 'Así es como se hace: \nPara compartir tu experiencia e-ESO, puedes usar el hashtag #e_ESO.', "El video muestra cómo participar en una sesión de preguntas y respuestas en línea. El presentador explica que los participantes pueden enviar sus preguntas y comentarios a través de un botón Q&A. Se muestra una interfaz de usuario con un botón Q&A destacado, y se indica que los participantes deben seleccionar 'All Panelists' en el menú desplegable para enviar sus preguntas a todos los panelistas. También se menciona que los participantes pueden compartir su experiencia usando el hashtag #e_ESO.", 'Estoy muy agradecido a la Escuela Europea de Oncología porque a menudo siento que el equipo quirúrgico es un poco descuidado. Así que me gustaría felicitar a Eizo por esto y quizás, ¿sabes? tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez, tal vez,', 'La sesión de preguntas y respuestas en línea\nHaz clic en el botón de Preguntas y Respuestas para enviar tus preguntas y comentarios\nPara compartir tu experiencia e-ESO usa: #e_ESO', 'Sí, me gustaría que me dieras la última palabra. Lo hice intencionalmente para que las diapositivas fueran sobre el manejo del cáncer de mama en el Reino Unido, un lugar de alto ingreso.', 'Y', 'E-sesion\nPregunta & Respuesta\nClick on the Q&A button\nto send your questions and comments\nTo share your e-eso experience use:\n#e_ESO', 'OK, yendo a compartir mis pensamientos con ti y los participantes\nE-sesión\nPregunta y Respuesta Sesión\nClick on the Q&A button\nto send your questions and comments\nTo share your e-eso experience use:\n54', "I'm sure that I'm not offending anybody, but I hope that I'm not offending anybody, but the story goes that when I first came to this country in 1999, breast surgery was dealt with by the most clumsy surgeons. They were perhaps not very skilled for visceral surgery, so they were relegated to subcutaneous cancer surgery.", 'Y y, y los resultados de los resultados fueron muy, muy malos.', 'Y en esos días, todavía sufríamos de los egos que, sabes, los cirujanos pueden hacer todo, o no necesitas, he aquí, he curado, tú no necesitas ningún chemo, tú no necesitas ningún radio, tú solo vuelve a mí en seis meses, y en un año, y estás saluda', 'Ahora, felizmente, ha habido un deseo, una decisión de establecer un servicio dedicado de cirugía de pecho', 'El audio dice que los viejos ya se han retirado y ahora hay más jóvenes que realmente se enfocan en el pecho.', 'Y', 'Y también, profesor Adiesio, ¿no es cierto que ha habido un establecimiento de centros de cáncer en un nivel regional que no solo para el cáncer de mama, sino para otras enfermedades, ha mejorado la calidad de la atención?', 'Sí, definitivamente así. Esto es particularmente cierto para el tratamiento de cáncer avanzado, como el tratamiento de cáncer de hígado, cáncer de páncreas, sarcomas, enfermedad peritoneal. Hay no punto en tener 20 unidades a lo largo del país. Tenemos 4 unidades acreditadas, totalmente auditas y de manera muy profesional. Eso es el escenario.', 'The presenter is explaining how to use the Q&A feature during an e-session. They are pointing out the Q&A button on the screen and instructing participants to click on it to send their questions and comments. The presenter also mentions that participants can share their e-ESO experience using a specific hashtag.', 'Y', 'Y e estamos mejorando en esto y los resultados están ahí. Los resultados de los resultados de la enfermedad han mejorado.', 'No puedo asegurarme de que pueda citarlo debido a la formación de un equipo multidisciplinario, pero es cierto que esta filosofía cohesiva y armonicada de abordaje a los pacientes con cáncer ha mejorado de manera significativa los resultados.', 'Sí, y creo que el punto principal, la síntesis de todo de esta charla es que los cirujés tienen que jugar un papel en este campo que se está desarrollando rápidamente y que tristemente, millones de personas no están obteniendo acceso a cuidados seguros, asequibles y oportunos.', 'Y', 'Y', 'Estoy muy agradecido, profesor Bash, por tu tiempo, por tu visión sabia y por compartir tus pensamientos conmigo.', '¡Solo quiero decirte gracias muchas gracias, gracias, Ciao Ciao, arrivederci, gracias, gracias! Gracias a profesor Barcio y profesor Audisio.', 'El próximo e-ESO Sesión tomará lugar el próximo jueves, julio 20º, 2017, a la misma hora e-SESISIÓN 405 Highlights of ICML 2017 Expert: Prof. Stefano Luminari, Universidad de Modena y Reggio Emilia, Modena, Italia Discusant: Dr. Astrid Pavlovsky, Clinical Research Center - FUNDALEU, Buenos Aires, Argentina Thank you! for participating to this e-session For additional information, please visit www.e-eso.net To share your e-eso experience use: #e_ESO', 'El próximo e-ESO Sesión tomará lugar el próximo jueves, julio 20º, 2017, en el mismo tiempo e-SESION 405 Highlights of ICML 2017 Expert: Prof. Stefano Luminari, Universidad de Modena y Reggio Emilia, Modena, Italia Discusant: Dr. Astrid Pavlovsky, Clinical Research Center - FUNDALEU, Buenos Aires, Argentina Thank you! for participating to this e-session For additional information, please visit www.e-eso.net To share your e-eso experience use: #e_ESO', 'Para entrar a la sección CME de la página, por favor cierre este cuadro de ventana y se llevará a cabo a su llegada a la sección e-ESO.', 'El siguiente e-ESO Sesión tomará lugar el próximo jueves, julio 20º, 2017, en el mismo horario e-Session 405 Highlights of ICML 2017 Expert: Prof. Stefano Luminari, University of Modena and Reggio Emilia, Modena, Italy Discussant: Dr. Astrid Pavlovsky, Clinical Research Center - FUNDALEU, Buenos Aires, Argentina Thank you! for participating to this e-session For additional information, please visit www.e-eso.net To share your e-eso experience use: #e_ESO', 'El próximo e-ESO Sesión tomará lugar el próximo jueves, 20 de julio, 2017, a la misma hora e-SESión 405 Highlights of ICML 2017 Expert: Prof. Stefano Luminari, Universidad de Modena y Reggio Emilia, Modena, Italia Discusant: Dr. Astrid Pavlovsky, Clinical Research Center - FUNDALEU, Buenos Aires, Argentina Thank you! for participating to this e-session For additional information, please visit www.e-eso.net To share your e-eso experience use: #e_ESO', 'Gracias por participar, y tenga un agradable día, noche, o cualquier momento. Adiós.']
,['Buenos días, buenas tardes, buenas noches.', 'Les habla la presentadora de la conferencia.', 'La Escuela Europea de Oncología les da la bienvenida a su sesión virtual 404.', 'La sesión virtual de hoy está dedicada al impacto de la cirugía oncológica en los resultados.', 'Le recordamos que esta actividad está acreditada por el CME.', 'Al final de la presentación, al cerrar la ventana de retransmisión, se le redirigirá a la evaluación de CME y al test de opciones múltiples.', 'Se pedirá a los Mastermind competitors que empiecen el test una vez finalizada la evaluación.', 'Como recordatorio durante la sesión en directo, todos los participantes pueden formular preguntas en directo en cualquier momento, simplemente pulsando el botón correspondiente en la parte superior de la página.', 'Esto es un extracto de la política que se publica en nuestro sitio web en su totalidad.', 'Las preguntas serán recibidas por el profesor Riccardo Audisio del Western Hospital, Universidad de Liverpool, Prescott, Reino Unido, quien las comentará con el profesor Charles M. Balch del MD Anderson Cancer Center de la Universidad de Texas, Houston, Texas, EE. UU.', 'Ahora le concedemos la palabra al Profesor Balch.', 'Profesor, su micrófono está abierto.', 'Hola a todos.', 'Soy Charles Balch de Houston, Texas, y me complace unirme a este seminario web para hablar sobre el impacto de los resultados quirúrgicos oncológicos y también participo en esto con mi buen amigo el profesor Riccardo Audisio, ex presidente de la Sociedad Británica de Oncología Quirúrgica y el presidente electo de la Sociedad Europea de Oncología Quirúrgica y él moderará una ronda de preguntas y respuestas al final de mi presentación.', 'Así que voy a exponer brevemente mis diapositivas, pero espero que más adelante descarguen varios artículos que creo que son documentos fundamentales que involucran cirujanos de todo el mundo, y que provienen de la Comisión Lancet que se publicaron en 2015 en Lancet Oncology.', 'El primero de ellos fue sobre cirugía global en el año 2030 y voy a leer unas citas de estos artículos.', 'La primera es que el acceso a la atención quirúrgica es esencial para reducir la mortalidad y morbilidad por afecciones quirúrgicas, sin embargo y esto es realmente un hecho importante y alarmante.Más de 2000 millones de personas no tienen acceso a la cirugía y la anestesia y, además, esta cifra está probablemente subestimada.', 'Por lo tanto, el acceso universal a una atención quirúrgica segura y asequible es fundamental.', 'En un artículo posterior en el que participamos el Dr. Audisio y yo, en Lancet Oncology, nos centramos en la cirugía oncológica global y en cómo administrar una cirugía oncológica segura, asequible y oportuna y realmente yo recomendaría que todos los que nos están escuchando lo leyeran.', 'Bueno, este artículo fue publicado en Lancet Oncology en 2015 y como pueden ver en las flechas el Dr. Audisio y yo fuimos sus coautores y parte del resumen que voy a destacar muy brevemente indicaba que para el año 2030, para lo que solo faltan trece años, calculamos que serán necesarios 45 millones de procedimientos quirúrgicos en todo el mundo y sin embargo menos del 25% de los pacientes de cáncer en todo el mundo realmente consiguen una cirugía segura, asequible u oportuna.', 'Así que queda mucho trabajo por hacer a este respecto y citaré algunas cosas más de este artículo en Lancet Oncology; que los problemas del acceso a la cirugía, la escasez de inversión en sistemas quirúrgicos públicos, la baja inversión en investigación, en formación, en cirugía y las brechas en formación están notablemente extendidas en todo el mundo.', 'Y hemos ofrecido soluciones que incluyen, un sistema público mejor regulado, especialmente en la formación y la certificación de cirujanos, colaboraciones internacionales, súper centralización de los servicios quirúrgicos y, volveré a hablar de esto más tarde, nuevos ensayos clínicos quirúrgicos, y nuevos enfoques para mejorar la calidad y el escalamiento de los sistemas de cirugía oncológica cáncer a través de la formación y la capacitación.', 'Les daré algunos ejemplos más adelante en esta conferencia.', 'Continuando con algunas declaraciones más resumidas de este importante artículo, lograr buenos resultados en la cirugía oncológica moderna es algo técnicamente exigente y es sabido que los resultados mejoran cuando dicha cirugía es realizada por equipos especializados en centros de alto volumen.', 'Y voy a entrar en esta importante estrategia con más detalle, pero haciendo hincapié en que estamos hablando de los pacientes más complicados, las operaciones más complicadas, y en centros de alto volumen.', 'Pero creo que lo que es importante como resumen es que el acceso a la formación de alta calidad en estas unidades, con la acreditación y el control de calidad adecuados para los aspirantes a cirujanos oncológicos, todavía tiene que ser cumplido formalmente por parte de las autoridades regulatorias en un número considerable de países.', 'Y aunque los países con niveles de renta altos como muchos países de Europa y Estados Unidos están impulsando una mayor especialización, a nivel mundial se necesitan cirujanos con una formación mucho más amplia; las asociaciones profesionales quirúrgicas deben desempeñar un papel más importante en esta área y, por último, que los países con niveles de renta altos y medios tienen el deber y la oportunidad de ampliar sus ofertas formativas de cirugía oncológica a países con niveles de renta bajos a través de intercambios bilaterales, de un mayor uso de la formación reforzada por la tecnología y colaboraciones para desarrollar contenidos curriculares específicos sobre el cáncer dentro de programas de formación de residentes de cirugía general.', 'Estos son solo algunos ejemplos de las asociaciones y la disponibilidad de programas globales de formación, como la Sociedad de Oncología Quirúrgica, la Sociedad Europea, la Asociación Británica y otras muchas organizaciones como la ESMO, la ASCO, y un grupo en el que participo llamado Health Volunteers Overseas, con sede en los Estados Unidos.', 'En las siguientes diapositivas quiero destacar la importancia y el valor de la especialización en términos de calidad de la atención.', 'Ha habido avances rápidos y sustanciales en la investigación del cáncer, con nuevos diagnósticos mediante biomarcadores, nuevas terapias sistémicas que incluyen quimioterapia, terapia hormonal, terapia dirigida y, más recientemente, inmunoterapia.', 'Y esto ha dado lugar a nuevas combinaciones y secuencias de tratamientos contra el cáncer, incluidos tratamientos neoadyuvantes y adyuvantes en los pacientes quirúrgicos.', 'Así que todos los que nos dedicamos a la cirugía tenemos que estar al día en los rápidos avances de estos campos.', 'También se están produciendo grandes avances en las tecnologías quirúrgicas en el quirófano, como la cirugía laparoscópica, la cirugía robótica y las técnicas de imagen intraoperatorias.Y entonces, la pregunta es, cómo podemos extirpar el cáncer con mejores resultados y más seguros, con menor incidencia de complicaciones y mejor control local regional, y la importancia, además de la rehabilitación y el restablecimiento de la función de los pacientes después de la cirugía.', 'Y ahora hay muchos cambios en la forma de administrar atención oncológica a través del concepto regional de centros oncológicos, un equipo multidisciplinario de especialistas oncológicos que tiene que incluir cirujanos que deciden pasar parte de su tiempo como miembros de un equipo multidisciplinario y participar en la planificación del tratamiento con nuestros colegas en radiooncología médica, patología y radiología.', 'Y les mostraré algunas diapositivas que demuestran que los centros oncológicos de alto volumen y los especialistas quirúrgicos obtienen mejores resultados al tratar cánceres complejos.', 'Y como ya he dicho, en el fondo esto no significa que para las operaciones de rutina y los pacientes de rutina sea necesario especializarse porque necesitamos cirujanos generales que practiquen cirugía de alta calidad y segura, pero que tengan el criterio necesario para saber cuándo algunos pacientes deben ser transferidos a un centro regional donde podrían obtener mejores resultados.', 'Y sólo para mostrarles un ejemplo de una enfermedad frecuente, me referiré a una publicación en el British Journal of Cancer en 2004, en la que comento que las pacientes con cáncer de mama tratadas por especialistas en comparación con una unidad no especializada tenían la mitad de riesgo de recibir un tratamiento inadecuado de la mama y un riesgo cinco veces menor de someterse a una cirugía axilar inadecuada y un riesgo nueve veces menor de recibir un tratamiento axilar inadecuado; y que las tasas de recurrencia locales, que es una medida del resultado quirúrgico, era un 57% menor a los ocho años y el riesgo de muerte por cáncer de mama era un 20% menor en las mujeres tratadas en unidades especializadas.', 'Y que concluimos que el manejo quirúrgico adecuado de los centros de la mama es fundamental para mejorar el desenlace del cáncer de mama, independientemente de dónde se haga.', 'Recuerden que esto se publicó en 2004 y desde entonces el sistema sanitario británico ha desarrollado centros oncológicos mucho más grandes y una regionalización del cáncer de mama sobre la que quizá el profesor Audisio quiera comentar.', 'Pero aquí en los Estados Unidos también hubo varios artículos en ese momento que mostraban que el volumen quirúrgico afecta la supervivencia si los cirujanos realizan al menos quince o más operaciones de cáncer de mama al año y que con la especialización quirúrgica (aquellos entrenados en cirugía oncológica de la mama u oncología quirúrgica) hubo una reducción del 36% en el riesgo de muerte a los cinco años incluso después de ajustar según el volumen hospitalario, el volumen quirúrgico, la edad, el estadio y la raza de la paciente.', 'Y he aquí algunos ejemplos que he seleccionado para el New England Journal of Medicine —estudios clásicos que muestran la relación entre el volumen hospitalario y la mortalidad quirúrgica en los Estados Unidos y nos muestra que para la colectomía, una operación más estándar, no hubo una diferencia comparando el volumen hospitalario y la mortalidad ajustada sin embargo, hubo una diferencia significativa en la mortalidad relacionada con el tratamiento después de la cirugía en el caso de la gastrectomía, esofagectomía y resecciones pancreáticas.', 'Continuando, en otro artículo que analiza el volumen del cirujano, el último artículo se centraba en el volumen del hospital.El volumen del cirujano y la mortalidad quirúrgica, de nuevo hubo algunas operaciones como, atendiendo a la mortalidad quirúrgica ajustada, no hubo mucha diferencia en la resección en casos de cáncer de pulmón o la cistectomía, pero hubo una diferencia muy significativa al comparar el volumen del cirujano y la mortalidad quirúrgica en la esofagectomía y las resecciones pancreáticas.', 'Esta diapositiva es un paréntesis, sólo para recordarles que sus opiniones son importantes, pueden hacer preguntas y enviar comentarios haciendo clic en el botón Preguntas y respuestas y enviando sus preguntas para que al final de esta conferencia el profesor Audisio y yo podamos moderar un debate con sus preguntas.', 'Volviendo a mi ponencia, quería señalar que el volumen del hospital o del cirujano no siempre es lo más importante, que los factores de riesgo del paciente también son importantes y he mostrado algunas fotos de mis viajes a África o incluso de mi propia experiencia personal de pacientes que han descuidado sus tumores o que han recibido un tratamiento inadecuado al principio y sufren al final metástasis regionales extensas que podrían haberse controlado si la cirugía hubiera sido más completa o se hubiera realizado en un momento anterior en el curso de la enfermedad del paciente.', 'Esto sirve para mostrar que el estadio clínico y la comorbilidad de los pacientes, especialmente entre los pacientes de edad avanzada, también marcarán una diferencia en el resultado.', 'Así que como resumen de algunos de estos artículos del New England Journal of Medicine y de todas las publicaciones de otras publicaciones, si se compara la mortalidad postoperatoria en centros especializados, en comparación con los hospitales locales de los Estados Unidos, no hubo diferencia en diversos tumores que afectan a la mama, melanoma, ovario, próstata, tiroides y útero.', 'Sin embargo, se encontraron diferencias entre los pacientes de alto riesgo al comparar los centros especializados con los hospitales locales, lo que muestra que para los pacientes de máximo y alto riesgo hubo una diferencia significativa en las especializaciones, mientras que en los pacientes de bajo riesgo no hubo diferencia entre los centros especializados y los hospitales locales.', 'Y por último, hubo una diferencia en todos los pacientes, en estas dos áreas, páncreas y esófago, al comparar la mortalidad postoperatoriaLo que demuestra que independientemente de los factores de riesgo de los pacientes, los pacientes tratados en centros especializados presentaron una reducción significativa en la mortalidad postoperatoria en estas dos afecciones.', 'Y he aquí otro artículo del Dr. Birkmeyer en el New England Journal of Medicine, que muestra de nuevo la mortalidad hospitalaria, basada tanto en el volumen del cirujano como en el volumen hospitalario, que muestra diferencias muy significativas al relacionar volumen y resultado con respecto a la mortalidad relacionada con la cirugía a los 30 días.', 'El Colegio Estadounidense de Cirujanos ha lanzado en un programa NSQIP un programa nacional de mejora de la calidad quirúrgica, para el cual la mayoría de los hospitales de los Estados Unidos participan en estos programas NSQIP y aquí solo se muestra cuando un programa fue iniciado a nivel nacional y en este caso por el Colegio Estadounidense de Cirujanos, que el 66% de los hospitales, ahora estos son casi todos los hospitales de los Estados Unidos, incluidos los hospitales locales, que el 66% de ellos mejora la mortalidad, el 82% mejora la morbilidad, los peores resultados, es decir, los cirujanos de bajo volumen u hospitales de bajo volumen, tenían más probabilidades de mejorar, pero incluso los que tenían buenos resultados también mejoraron.', 'Y que el número de valores atípicos bajos aumentó y el número de valores atípicos altos disminuyó, lo cual es bueno tomando como base estos programas de calidad iniciados a nivel nacional.', 'Y gracias a este programa, se estimó que cada hospital evitaba 250 complicaciones por año en cada hospital en promedio.', 'De nuevo, quiero que envíen sus preguntas haciendo clic en el botón de Preguntas y respuestas y luego continuaremos en otra área sobre la formación de cirujanos oncológicos.', 'Y aquí hay un punto muy importante, aunque hablamos de oncología quirúrgica y súper especialización, la Junta Americana de Cirugía, que certifica a todos los cirujanos generales y especialidades basadas en cirugía general, incluye la oncología quirúrgica incluyendo la cirugía de cabeza y cuello entre las áreas centrales que se incluyen en sus exámenes de certificación.', 'Por lo tanto, la oncología quirúrgica y el conocimiento del papel de la cirugía en el manejo del cáncer son igualmente importantes para las quemaduras traumáticas, la pediatría vascular, la medicina intensiva y otros criterios.', 'Y yendo más allá, hay dos organizaciones, la Sociedad de Oncología Quirúrgica y la Sociedad Europea de Oncología Quirúrgica, dirigida por el Dr. Chandra Are, aquí en los Estados Unidos, pero incluyendo también al profesor Audisio representando a la ESSO, que publicaron dos importantes artículos el primero mostraba grandes variaciones en la formación de oncólogos quirúrgicos en todo el mundo, y luego proponiendo conjuntamente un plan de estudios global en oncología quirúrgica que puede ser utilizado tanto para la formación de cirujanos generales, como también como recomendaciones curriculares para la formación de especialistas en oncología quirúrgica, que son muy importantes a nivel nacional para el manejo de pacientes muy complejos y para dirigir los programas de formación de cirujanos generales.', 'Y nuestras características curriculares, tanto en los Estados Unidos como en este plan de estudios global, incluyó las características distintivas de los oncólogos quirúrgicos, que abordan operaciones complejas, que forman parte de un equipo multidisciplinario que involucraba la oncología médica y radioterápica en la planificación del tratamiento de los pacientes desde el inicio de su diagnóstico y que participan como oncólogos en el manejo a largo plazo del paciente con cáncer.', 'Y, recientemente, la Junta Americana de Cirugía ha aprobado un nuevo certificado en oncología quirúrgica general compleja, reconociendo que hay una cantidad suficiente de material especializado que justifica una subcertificación independiente en oncología quirúrgica general compleja.', 'Ahora, en un nivel formal, y esto se une a las certificaciones de la Junta en Oncología Médica, Oncología Pediátrica y Oncología Radioterápica, ahora hay una Junta separada para oncología quirúrgica general compleja.', 'Y esta formación de dos años implica una formación adicional en atención multidisciplinaria del cáncer, manejo quirúrgico de enfermedades oncológicas, asesoramiento del paciente sobre prevención e intervención del cáncer y, muy importante, la participación y el liderazgo en investigación clínica y diseño de ensayos clínicos.', 'Y esto incluye cosas que realmente todos los cirujanos generales que tratan el cáncer de mama, por ejemplo y especialmente los oncólogos quirúrgicos que se especializan en cáncer de mama, necesitan saber, cómo usar la ecografía y la biopsia con aguja gruesa.', 'Y digo esto porque en mis viajes a otros países todavía hay muchos centros locales en los que hacer una biopsia abierta, para hacer el diagnóstico del cáncer de mama es el procedimiento habitual.', 'Mientras que en los hospitales especializados y en muchos centros de Europa y Estados Unidos, incluso a nivel local, el procedimiento de referencia consiste en utilizar una biopsia con aguja gruesa en lugar de una biopsia abierta, como tratamiento de elección para las pacientes con un primer diagnóstico de cáncer de mama.', 'Ahora en las siguientes diapositivas, quiero enfatizar lo que es una parte muy importante de las pacientes quirúrgicas, aquí en el cáncer de mama en pacientes que presentan enfermedad en estadio tres, que con quimioterapia simultánea y terapia sistémica y dirigida, las pacientes están logrando una respuesta patológica completa en el 40% al 60% de estas pacientes, y esto va a cambiar la forma de tratar a las pacientes quirúrgicamente porque la cirugía puede no ser la primera modalidad de tratamiento.', 'Aquí, en el cáncer de mama en los Estados Unidos, la mayoría de las pacientes con cáncer de mama en estadio dos y estadio tres y creo que en muchos centros de Europa, especialmente para el estadio tres, la terapia sistémica es el primer tratamiento y luego la cirugía es el segundo tratamiento, así que hay muchas planificaciones diferentes que tenemos que hacer con nuestros colegas de oncología médica con respecto al momento de la cirugía, el papel del ganglio centinela que es diferente en este contexto; pero esto representa el futuro de nuestros tratamientos para muchas enfermedades para las que el cáncer de mama es el prototipo que cambiará.', 'Pero aquí hay otro ejemplo, un paciente con melanoma avanzado con gran masa tumoral e irresecable en la ingle y la pelvis, que fue tratado con inmunoterapia y después de la inmunoterapia tuvo una reducción tal que facilitó la operación, y de hecho en este paciente, cuando se extrajeron todas estas masas (todas contenían inflamación crónica y cicatriz), este paciente tuvo una respuesta patológica completa que no se habría podido lograr sin la resección quirúrgica de las masas tumorales que originalmente se detectaron.', 'Así que el futuro de la terapia sistémica para el cáncer está cambiando drásticamente, hay terapias más efectivas y de hecho muchas de las nuevas terapias van a ser terapias orales que son mucho menos tóxicas y van a repercutir en el paciente quirúrgico.', 'A continuación se presenta un ejemplo de los nuevos fármacos orales o subcutáneos disponibles para diversos tipos de cáncer.', 'Solo para darles algunos ejemplos, estos son los ejemplos más drásticos, pero se están volviendo más típicos.', 'Se trata de un paciente que tenía un melanoma metastásico extenso que incluso después de dos semanas de tratamiento se puede ver que la PET/TAC mostró reducciones drásticas de este tumor extenso.', 'Y estos agentes son tan potentes que ya se están empezando a aplicar en pacientes con melanoma en estadio tres.', 'El otro gran avance que está ocurriendo es el uso de inmunoterapia con los nuevos inhibidores de los puntos de control, que ahora actúan en más de 17 tipos diferentes de cáncer, y esto representa nuestro futuro que se utilizará cada vez más en pacientes quirúrgicos.', 'Pero sólo para darles algunos ejemplos individuales para que tengan una idea del futuro de la terapia, he aquí un ejemplo de una metástasis de melanoma muy grande, que después de una dosis única de inmunoterapia combinada este paciente tuvo una respuesta patológica completa, demostrada por la resección quirúrgica del tejido restante.', 'Y este es quizás un ejemplo más famoso, nuestro presidente estadounidense Jimmy Carter, esto es en la esfera pública, que presentó metástasis hepáticas y cerebrales. Y a los 92 años fue tratado durante un corto intervalo con una dosis única de un inhibidor del punto de control anti pd1.', 'Ahora lleva libre de la enfermedad y en remisión completa durante más de catorce meses, con este agente único en un paciente que es muy mayor y toleró este tratamiento.', 'Quería mostrar otro ejemplo más de un paciente con cáncer de cabeza y cuello que tenía 96 años, enfermedad extensa e incluso después de cuatro semanas con esta terapia anti pd1 única tuvo una respuesta drástica.', 'Así que, si vemos este tipo de respuestas al tratamiento en pacientes con enfermedad más avanzada, pueden estar seguros de que esto se llevará al tratamiento de pacientes con enfermedad más incipiente, como el cáncer en estadio dos y estadio tres.', 'Entonces, como cirujanos en ejercicio, ¿cómo vamos a mantenernos al día en estos campos de rápido avance, tanto en terapia sistémica, radioterapia como en los avances tecnológicos en el quirófano y los nuevos avances en los biomarcadores utilizados para seleccionar pacientes para estos diferentes tratamientos?', 'Y aquí, una vez más, esto está sucediendo de formas muy variadas, con reuniones en vivo, especialmente las realizadas por la SSO y la ESSO y BASO, pero también a través de afiliaciones con centros de atención especializada como en mi institución, con hospitales locales, con avances en revistas, con colaboraciones por correo electrónico o con seminarios web en vivo, como este seminario web que está llevando a cabo el Instituto Europeo de Oncología.', 'El Dr. Audisio y yo somos los redactores jefe de nuestras dos respetadas revistas y se las recomendamos, para mantenerse al día en este campo, tanto en la versión en línea como en la versión impresa.', 'La Sociedad de Oncología Quirúrgica también tiene ahora un programa de autoevaluación, que se lleva a cabo para ayudar a los cirujanos a mantenerse al día en el campo de la oncología.', 'Y en los Estados Unidos usamos cada vez más la telemedicina, que se integra cada vez más en las operaciones en curso de los departamentos de especialidades de hospitales, agencias de salud domiciliaria, consultorios médicos privados, y en algunos casos incluso con fines educativos en el hogar del consumidor.', 'Así, la telemedicina se usa cada vez más no sólo en Estados Unidos sino en todo el mundo, tanto para la formación médica de los médicos, especialmente de los de centros locales y de las zonas más rurales, como para los servicios de consulta especializada, sobre todo para ayudar en la toma de decisiones de atención por parte del paciente, en otros lugares o incluso en otros países.', 'La tecnología nos permite ahora ver con alta resolución imágenes de radiografías, imágenes de patología, e incluso hacerlo, ver en tiempo real imágenes de ecografía, que se están llevando a cabo en otro sitio y se están transmitiendo electrónicamente a un sitio central mediante telemedicina.', 'Incluso he participado en una compañía llamada Cancer Expert Now, soy el Presidente de Cancer Expert Now International, donde hacemos interconsultas con otros médicos.', 'Aquí se muestra un ejemplo mediante telemedicina, de uno de nuestros neurooncólogos haciendo una interconsulta con un médico en otro país sobre un paciente complicado con un tumor cerebral.', 'Y este es sólo otro ejemplo donde estuve en Huangshan, China, e hicimos un comité virtual de telemedicina sobre tumores aquí con la Dra. Hope Rugo, que estaba en San Francisco en ese momento, para hablar sobre una paciente complicada con cáncer de mama.', 'Y aquí está, justo después de esto, una videoconferencia en la misma ubicación, sobre un paciente complejo de cáncer de colon con el Dr. Glen Balch que es jefe de cirugía colorrectal en Emory en Atlanta.', 'Nuevamente mostrando las ilustraciones de cómo los comités virtuales sobre tumores pueden ser utilizados para ayudar en el asesoramiento y recomendaciones para pacientes con cáncer complejo, ahora en cualquier parte del mundo.', 'Y he incluido esta diapositiva, de un resumen que se presentó el año pasado en la ASCO, que muestra que el 91% de los asistentes a comités virtuales sobre tumores lo consideraron muy útil, y el 100% de los asistentes notaron que la calidad de la atención mejoraba.', 'Muy bien, y de nuevo estamos casi terminando nuestra charla, voy a hacer el resumen en breve.', 'Y solo como recordatorio para utilicen los botones de Preguntas y respuestas a medida que resumo.', 'Entonces, como cirujanos ¿cómo nos adaptamos a un cambio en pacientes oncológicos que cambian rápidamente?', 'Está claro en los Estados Unidos y creo que en muchas partes de Europa y en otros lugares del mundo que muchos pacientes de cáncer recibirán más de una modalidad de tratamiento, y que la atención del cáncer contemporánea es un enfoque de equipo, que combina la sabiduría colectiva de cirujanos, oncólogos médicos, oncólogos radioterápicos y otras veces con su patología y oncología radioterápica.', 'Con los avances en la terapia sistémica eficaz es evidente que el cáncer se está convirtiendo en una enfermedad crónica o curable.', 'El cáncer de mama es un muy buen ejemplo de ello.', 'Y después de todos estos tratamientos intensivos, la supervivencia en pacientes que viven durante años después de su tratamiento están en riesgo de sufrir de segundas neoplasias primarias, cumplimiento del tratamiento y las consecuencias de su tratamiento intensivo multidisciplinario.', 'El campo de la oncología es una de las especialidades que más rápidamente avanza y cambia en toda la medicina, y eso significa que nosotros como cirujanos tenemos que mantenernos al día a través de la formación continua e incorporar estos cambios en el plan de estudios de nuestros alumnos.', 'Y también es importante validar estos avances en pacientes quirúrgicos sistemáticamente a través de ensayos clínicos.', 'Y como he tratado de enfatizar, hay una colaboración continua de organizaciones profesionales, especialmente en los Estados Unidos, en Europa, en la formación continua de médicos, que tiene que ocurrir permanentemente, y para mejorar el plan de estudios de los cirujanos generales y las especialidades de cirugía general en todas las instituciones del mundo.', 'Los cirujanos para lograr un mejor resultado van a tener que dividir a sus pacientes e identificar aquellos para los que la especialización o derivación a centros de alto volumen especializados en ciertas enfermedades tendrá un mejor resultado para el tratamiento de pacientes con cánceres complejos o avanzados.', 'Y existe una demanda pública e incluso gubernamental de obtener una atención óptima de especialistas, de centros oncológicos multidisciplinarios y de la regionalización de la terapia oncológica.', 'Sin embargo, es importante enfatizar que esto no significa que todos los pacientes con cáncer necesiten ser tratados por especialistas, especialmente si el diagnóstico se hace pronto y el tratamiento es estándar y no necesita múltiples especialidades para su atención.', 'Realmente creemos que los pacientes deben permanecer en su entorno local todo lo posible, y aquí es donde cosas como la telemedicina pueden desempeñar un papel para ayudar a garantizar que los pacientes, incluso en un hospital local más pequeño, reciban asesoramiento y planificación del tratamiento, con aportaciones de especialistas a través de la telemedicina o la interconsulta.', 'Y que quienes nos especializamos como oncólogos quirúrgicos, deberíamos ser líderes en formación e investigación entre la comunidad de cirugía general.', 'Ya casi termino con mi charla; en resumen, las sociedades profesionales, como la sociedad que represento, la Sociedad de Oncología Quirúrgica y antes fui Director Ejecutivo de ASCO, la SSO europea, el Colegio Americano de Cirujanos, la ASCO, son ejemplos de organizaciones que pueden ser componentes importantes para definir los estándares para tratar pacientes quirúrgicos con cáncer y proporcionar formación, capacitación, talleres para ayudar a otros cirujanos a lograr una atención óptima del cáncer.', 'Y que las asociaciones entre sus sociedades profesionales, que están bien desarrolladas entre los europeos en las organizaciones estadounidenses de SSO, mejorarán nuestra especialidad y la atención de nuestros pacientes.', 'Así que esa es mi conclusión, ahora voy a pasar el testigo al profesor Audisio para escuchar sus comentarios y para responder cualquier pregunta que puedan haber hecho durante esta exposición.', 'Muchas gracias.', 'Gracias Charles, no tenemos palabras para agradecerle al profesor Balch por resumir el presente y el futuro de la cirugía oncológica de buena calidad en todo el mundo.', 'Y realmente quiero decir todo el mundo porque no es sólo el procedimiento quirúrgico sofisticado, súper avanzado que se podría esperar de un centro de referencia de asistencia especializada avanzada, esta es una filosofía, esta es una actitud muy básica, que todos deberíamos adoptar incluso en un hospital general comarcal, hay espacio y hay necesidad de cierto tipo de cirujano, que sea consciente de la multidisciplinariedad de la práctica oncológica, y que fuera más propenso a hablar con diferentes especialistas, para luego ofrecer a sus propios pacientes el tratamiento más adecuado.', 'Entonces ¿estoy en lo correcto al decir, Profesor Balch, que el mensaje con el que hay que quedarse es que la cirugía oncológica de buena calidad es algo que se espera, que se demanda y que es una necesidad mundial no sólo de las principales unidades oncológicas, sino también de la todos los hospitales comarcales?', 'Eso es absolutamente cierto y creo que las organizaciones profesionales y los gobiernos de cada país tienen que insistir en que sus ciudadanos reciban atención de alta calidad y asegurar, en cada país, que haya cirujanos bien formados, que haya sistemas organizativos, como centros oncológicos regionales, y creo cada vez más que hay relaciones internacionales con personas u organizaciones, centros médicos y centros oncológicos, que pueden ser de ayuda para la interconsulta en pacientes complejos, y en la formación de los cirujanos en las comunidades locales e incluso en los centros oncológicos más provinciales o nacionales en los hospitales.', 'Por ejemplo, la semana que viene voy a formar un comité multidisciplinario sobre tumores con un hospital provincial de China, y parte del tema a tratar va a ser cómo podemos aumentar el uso de la videoconferencia para aumentar el acceso de los médicos en China con expertos para ayudar en la toma intelectual de decisiones para pacientes oncológicos complejos.', 'Creo que este es un tema muy candente y tal vez debamos dedicar un poco más de tiempo a pensar la mejor manera de facilitar esto.', 'Acabamos, acaba usted de señalar cómo esa red existe, y yo animaría a los jóvenes cirujanos en formación a que se sientan absolutamente libres de perseguir sus sueños, estudiar sus temas, y cuando se encuentren con el gran profesor en la reunión le tiren de la manga y, y, contacten con especialistas, especialistas de primer nivel, y que luego se sientan libres de comunicarse con ellos, tal vez entablen con ellos un debate y se sientan lo suficientemente cómodos como para aprovechar la gran red que está disponible a través de la SSO, la Sociedad Estadounidense de Oncología Quirúrgica, y la europea también; estamos trabajando en este momento para facilitársela a estos países en desarrollo en África, a través del fondo no restringido que las compañías farmacéuticas están poniendo a disposición, este es, en mi opinión, el camino a seguir.', 'Así que los animo a venir y hablar con el profesor Balch, conmigo mismo, y con quien quieran para mejorar sus conocimientos.', 'Creo que lo importante es que no podemos obligar a estos cambios, la iniciativa tiene que venir del cirujano en cada comunidad, de su hospital, y de su gobierno, tanto su gobierno local como su gobierno nacional, para elevar el nivel de expectativas para una atención de alta calidad y oportuna.', 'Y luego trabajar en cada país dentro de las limitaciones de sus recursos, para lograrlo con el tiempo.', 'Hagamos también alguna advertencia, algo fácil de entender sobre técnicas súper avanzadas, es decir, no quieres un escáner PET, cuando no tienes un ecógrafo.', 'Podrías no plantearte comprar un equipo robótico, cuando te falta un aparato básico de laparoscopia, y por lo tanto necesitas desarrollar el sistema, como usted dice, paso a paso, de acuerdo con las instalaciones locales a nivel regional o provincial.', 'Sí, es absolutamente cierto; he estado en un hospital importante en África que tiene una máquina de radioterapia muy cara que aún está en cajas porque no tienen los radiólogos o los técnicos en radiología para el componente de servicio, pero tienen la máquina en una caja, que alguien compró, pero no basta con eso.', 'Así que se necesita planificación a nivel gubernamental y hospitalario, para asegurarse de definir no sólo los gastos financieros en edificios y equipos sino también la ejecución de los servicios por individuos calificados y entrenados, a todos los niveles.', 'Así, Profesor Balch, siempre tenemos enemigos y aliados cuando queremos lograr un objetivo, y esa es una forma tonta y restrictiva de verlo, pero ha ocurrido que la mayoría de los oncólogos quirúrgicos culpen a los cirujanos generales por no adoptar una actitud o filosofía oncológica y, por otro lado, algunos oncólogos médicos están barriendo las habilidades de los cirujanos hasta esconderlas bajo la alfombra.', '¿Estamos en lo cierto al decir que realmente necesitamos a todos y les damos la bienvenida a todos a bordo, porque este es un debate en curso que es un placer crear juntos a medida que se desarrolla?', 'Estoy de acuerdo, y creo que el punto aquí es que el cirujano tiene que estar involucrado en la planificación del tratamiento.', 'Como usted y yo hemos comentado hay otras especialidades y cirujanos que pasan todo su tiempo en el quirófano, y permiten que el profesional con formación en medicina interna se ponga al frente y tome todas las decisiones y ahora en esa especialidad, y estoy hablando de cirugía cardíaca, el cirujano es el tribunal de último recurso, después de que todos los tratamientos médicos hayan fracasado.', 'Y me preocupa que si el cirujano no aporta la perspectiva quirúrgica a la planificación del tratamiento, y está involucrado con el oncólogo médico y el oncólogo radioterápico, los pacientes podrían no obtener todas las ventajas de un verdadero enfoque de tratamiento multidisciplinario que implique cirugía, pero estamos mejor entrenados y tenemos la experiencia para determinar cuál es el mejor momento de la cirugía en determinada combinación y secuencia.', 'Si no estamos sentados a la mesa, seremos el tribunal de último recurso después de que todo lo demás fracase con el tratamiento médico y la radioterapia.', 'Así que tenemos que participar en la planificación del tratamiento desde el principio.', 'Sí, bueno, una vez más adoptamos totalmente una actitud multidisciplinaria, donde cada vez más y más y quizás nuevas figuras están tomando parte, por ejemplo, pienso en radiólogos intervencionistas que hace tan sólo tal vez una o dos décadas eran casi desconocidos, pero hoy en día están las estenosis de endoprótesis vasculares, ellos colocan catéteres, administran medicación e isótopos, así que una vez más vamos a reiterar claramente la idea de que queremos a todos a bordo aquí.', 'Aprovecho el tiempo porque entiendo por parte del organizador que hay un problema con la recepción de las preguntas y este servicio no ha estado disponible hoy, hay un fallo técnico.', 'Así que nos disculparemos por esto y nos aseguraremos de que si tienen alguna pregunta puedan regresar, la audiencia puede contactar conmigo y con el profesor Balch por escrito y nos complacerá responderles o satisfacer cualquier solicitud.', 'Pero me complace mucho ser parte de esta importante sesión y unirme a usted, un oncólogo quirúrgico de renombre internacional, en este foro educativo por videoconferencia.', 'También estoy muy agradecido a la Escuela Europea de Oncología, porque a menudo sentimos que el equipo quirúrgico está un poco descuidado, así que me gustaría felicitar a la ESO por esto y quizás, en fin, retorcerles el brazo y asegurarse de que se otorgue más espacio y más interés a lo que están haciendo los cirujanos.', '¿Hay alguna nota concluyente que quiera destacar, profesor Balch?', 'Sí, me gustaría que me dieran la última palabra, mostré a propósito las diapositivas sobre el tratamiento del cáncer de mama en Gran Bretaña, una región de niveles altos de renta.', 'Y tengo entendido que después de esa época alrededor del año 2002-2004, el gobierno británico se puso manos a la obra para desarrollar los centros oncológicos, la regionalización y que la atención en el cáncer de mama ha mejorado enormemente.', 'Usted se mueve en ese ámbito y ha sido testigo de ello, ahora me pregunto si le gustaría comentar algo, porque una de las cuestiones importantes para nosotros es que se necesita cierto nivel de especialización, incluso en el cáncer de mama.', 'Está bien, voy a compartir mis opiniones con usted y los participantes.', 'Estoy seguro de que, espero no ofender a nadie, pero la historia dice que, cuando llegué a este país en 1999, la cirugía de mama estaba en manos de los cirujanos más torpes, quizás no estaban muy capacitados para una cirugía visceral, así que quedaban relegados a cirugías de cáncer subcutáneo.', 'Y por lo tanto, los resultados estéticos fueron terribles, y los resultados oncológicos también fueron muy, muy deficientes.', 'Y en esos días, todavía éramos presa de los egos, que usted sabe que los cirujanos lo pueden todo, o usted no lo necesita, yo le he curado, no necesita quimio, no necesita radio, solo regrese dentro de seis meses, de un año, y está arreglado.', 'Ahora, afortunadamente, ha habido un deseo, una decisión de crear cirujanos especializados en la mama.', 'Los viejos se han retirado y ahora hay cada vez más jóvenes que realmente se centran solamente en el cáncer de mama.', 'Y esto significa que no hacen guardias nocturnas, simplemente se centran en la cirugía del cáncer de mama.', 'También profesor Audisio, ¿no es cierto que se han establecido centros de oncología regionales que, no solo para el cáncer de mama, sino para otras enfermedades, ha elevado la calidad de la atención?', 'Sí, definitivamente, esto es especialmente cierto para la cirugía de cáncer avanzado, como señaló usted, la cirugía hepática, la cirugía pancreática, los sarcomas, la enfermedad peritoneal; no tiene sentido tener veinte unidades en todo el país, tenemos cuatro unidades acreditadas totalmente auditadas, y muy bien controladas y con un rendimiento maravilloso, así que ese es el escenario.', 'Pero el punto que quería dejar claro es que creo que la generación joven está aceptando, como decía usted antes, la idea de trabajar juntos y decidir: sí, esto es una cuestión quirúrgica, pero ¿en qué momento? ¿en qué pacientes? ¿cuán agresivos deberíamos ser?', '¿Y qué queda después de la cirugía?', 'Y, definitivamente, estamos mejorando en esto y los resultados están ahí, los resultados del cáncer han mejorado.', 'No estoy seguro de que pueda citarlo debido a la, esto, la creación del equipo multidisciplinario, pero es indudablemente cierto que esta filosofía cohesionada, este abordaje armónico de los pacientes con cáncer ha mejorado extraordinariamente los resultados.', 'Sí, y creo que la enseñanza, el resumen de toda esta charla es que los cirujanos tienen que desempeñar un papel en este campo de rápido avance y que lamentablemente miles de millones de personas no tienen acceso a una atención segura, asequible y oportuna.', 'Y creo que todos nosotros tenemos que intensificar nuestros esfuerzos para ayudar a los pacientes de cáncer de todo el mundo, para que, independientemente de dónde vivan, puedan tener acceso, ya sea a través de un médico local más formado y capacitado o mediante la derivación a un centro regional.', 'Y que el proceso formativo se mejore en todos los países para que todos podamos estar al día en este campo en rápido crecimiento y podamos introducir de manera segura estas nuevas terapias, a medida que estén aprobadas por los gobiernos y los planes de seguros.', 'Le estoy muy agradecido Profesor Balch por su tiempo, por su sabia visión y por compartir sus pensamientos con nosotros.', 'Solo quiero decirles grazie molte, grazie, ciao ciao, arrivederci, grazie Gracias Profesor Balch y gracias Profesor Audisio.', 'Damas y caballeros, la sesión ha concluido.', 'Si están interesados en la acreditación del CME, los participantes deben cumplimentar el formulario de evaluación y pasar la prueba de múltiples opciones del CME.', 'Para ingresar a la sección del CME del sitio, por favor cierre esta ventana y se le redirigirá a los formularios del CME.', 'Si usted es un Mastermind competitor, ingresará al test después de la evaluación.', 'La próxima sesión de e-ESO tendrá lugar el 20 de julio a la misma hora y el profesor Stefano Luminari de la Universidad de Modena y Reggio Emilia, Modena, Italia, debatirá con la Dra. Astrid Pavlovsky del Centro de Investigación Clínica Fundaleu, Buenos Aires, Argentina, sobre el siguiente tema Aspectos principales del ICML 2017.', 'Gracias por participar y que tengan una agradable tarde, día o noche.']
,['Good morning, afternoon and evening.', 'This is the conference operator.', 'The European School of Oncology welcomes you to their four hundred and forth e-session.', "Today's e-session is the impact of oncological surgery on the outcomes.", 'Please note that this activity is CME accredited.', 'At the end of the presentation by closing the webcast window you will be directed to the CME evaluation and multiple choice test.', 'Mastermind competitors will be prompted to start the quiz after having completed the evaluation.', 'As a reminder during the live session all participants can ask questions live at any time by simply pressing on the relevant button at the top of the page.', 'Here is an extract of the policy which is published on our website in full.', 'Questions will be received by Professor Riccardo Audisio of the Western Hospital University of Liverpool, Prescott, United Kingdom who will discuss them with Professor Charles M. Balch of the University of Texas MD Anderson Cancer center, Houston, Texas, USA.', 'We will now hand over to Professor Balch.', 'Professor, your line is now open.', 'Hello everyone.', "This is Charles Balch from Houston, Texas and I'm pleased to join this webinar to talk about the impact of oncologic surgical outcomes and I'm also involved in this is my good friend Professor Riccardo Audisio, the past president of the British Society of Surgical Oncology and the president elect of the European Society of Surgical Oncology and he will lead a question and answer period at the end of my presentation.", "So I'll go briefly through my slides but I'm hoping that you will download at a future time several papers that I think are seminal papers involving surgeons around the world, and these come from the Lancet Commission that were published in 2015 in Lancet Oncology.", "The first of which was about global surgery in the year 2030 and I'll just give a few quotes from these papers.", 'The first is access to surgical care is essential for reduction of mortality and morbidity from surgical conditions, however and this is really an important fact and a startling one. More than two billion people have no access to surgery and anesthesia and moreover this figure is probably an underestimate.', 'So universal access to safe, affordable, surgical care is essential.', 'In a subsequent paper that Dr Audisio and I were involved in, in Lancet Oncology focused on global cancer surgery and how to deliver safe, affordable and timely cancer surgery and I would really commend this paper for all of you who are listening to read.', "So this paper was published in Lancet Oncology in 2015 and as you can see from the arrows after Audisio and I were both co-authors of this and part of the summary which I will highlight very briefly stated that by the year 2030, that's just thirteen years from now, we estimate that forty five million surgical procedures will be needed worldwide and yet less than twenty five percent of cancer patients worldwide actually get safe affordable or timely surgery.", "So there's much work to be done here and just a few more quotes from this paper in Lancet Oncology that issues of access to surgery, a paucity of investment in public surgical systems, low investment in research, in training, in surgery and educational gaps are remarkably widespread globally.", "And we offered up solutions that include, a better regulated public system especially in the training and certification of surgeons, international partnerships, super centralization of surgical services and I'll come back to that later, novel surgical clinical trials, and new approaches to improve quality and scale-up of cancer surgical systems through education and training.", "And I'll give some examples of that later in this lecture.", 'So going on with a few more summary statements from this important paper, achieving good outcomes in modern cancer surgery is technically demanding and it is recognized that outcomes improve when such surgery is undertaken by specialized teams in high volume centres.', "And I'll go into this important strategy in more detail but with the emphasis that we're talking about the more complicated patients, the more complicated operations, and in high volume centers.", "But I think what's important as a summary is that access to high-quality training in these units with suitable accreditation and quality control for aspiring oncologic surgeons has yet to be formally met by regulatory authorities in actually quite a number of countries.", 'And although high-income countries such as many countries in Europe and in the United States are driving greater specialization, globally there is a need for surgeons with much wider training the surgical professional societies must play an enhanced role in this area and finally, that high income and middle income countries have a duty and an opportunity to expand their educational offerings on cancer surgery to low income countries through bilateral exchanges, a greater use of technology enhanced learning, and partnerships to develop specific curriculum content on cancer into general surgery residency training programs.', "These are some examples of the partnerships and availability of global programs in education including the Society of Surgical Oncology, the European Society, the British Association and several other organizations such as ESMO, ASCO, and a group that I'm involved in called Health Volunteers Overseas, that's based in the United States.", 'In these next few slides I want to emphasize the importance and the value in terms of quality care on specialisation.', "There's been a rapid and substantial advances in cancer research, with new diagnostics in biomarkers, new systemic therapies including chemotherapy, hormone therapy, targeted therapy and more recently immunotherapy.", 'And this has led to new combinations and sequences of cancer treatments including new adjuvant, and adjuvant therapies in the surgical patients.', 'So all of us in surgery has to keep up with these rapidly advancing fields.', 'There are major advances in surgical technologies in the operating room as well, such as, laparoscopic surgery, robotic surgery and intraoperative imaging techniques. And the question is then, how can we excise cancer with better and safer results, with lower incidence of complications, and better local regional control, and the importance in in addition of rehabilitation and restoring function to patients after surgery.', 'And there are many changes now in the delivery of cancer care through the regional concept of cancer centers, a multi-disciplinary team of oncologic specialists which have to include surgeons who choose to spend part of their time as part of a multi-disciplinary team and participating in treatment planning with our colleagues in medical radiation oncology, pathology and radiology.', "And I'll show just a few slides of evidence that high volume cancer centers and surgical specialists have better outcomes for treating complex cancers.", 'And as I mentioned again at the end of, this does not mean for the routine operations and routine patients that need to specialize because we do need general surgeons who practice high-quality and safe surgery but has the judgment to know when some patients should be passed on to a regional centre where they could get better results.', 'And just to show you an example for a common disease, this was a publication in the British Journal of Cancer in 2004, in which I quote that breast cancer patients treated in by specialists compared to non specialist unit had half the risk of inadequate treatment of the breast and a five fold lower risk of inadequate axillary surgery and a nine times lower risk of inex inadequate axillary treatment. and that actually local recurrence rates which is a metric of surgical outcome was fifty seven percent lower at eight years and the risk of death from breast cancer was twenty percent lower for women treated in specialist units.', 'And that we conclude that adequate surgical management of breast centers is fundamental to improving the outcome for breast cancer irrespective of where it is delivered.', 'Now remember this was published in 2004 and since then the British healthcare system has really developed much greater cancer centres and regionalisation of breast cancer which perhaps Professor Audisio may want to comment on.', 'But here in the United States there were also a number of papers at the time showing that surgical volume impacts survival if surgeons perform at least fifteen or more breast cancer operations per year and that with surgical specialisation those trained in breast surgery oncology or surgical oncology there was a thirty six percent reduction in the risk of death at five years even after adjusting for hospital volume, surgical volume, age, stage and race of the patient.', "And here are just a few examples that I selected for the New England Journal of Medicine classic studies showing the relationship between hospital volume and surgical mortality in the United States and the teaching point here is for colectomy a more standard operation there wasn't a difference comparing hospital volume and adjusted mortality however, there was a significant difference in treatment related mortality after surgery for gastrectomy a suffragette to me and pancreatic resections.", 'Going on in another paper looking at surgeon volume the last paper was on hospital volume. Surgeon volume and operative mortality, again there were some operations such as, were looking at adjusted operative mortality there was not much different in resection for lung cancer or for cystecomy, but there was a highly significant difference comparing surgeon volume and operative mortality for esophagectomy and pancreas resections.', 'Now this is an interlude slide just to remind you that your views are important, you can ask questions and send comments by clicking on the Q & A button and sending in your questions so at the end of this lecture Professor Audisio and I can lead a discussion with your questions.', "Getting back to my lecture I wanted to point out that volume either hospital or surgeon volume, isn't always the most important thing, that patient risk factors are important as well and I've showed some photos either from my travels in Africa or even from my own personal experience of patients who have neglected their tumors or they've had inadequate treatment at the beginning and suffer at the end with extensive regional metastasis that might have been controlled had surgery been more complete or done at an earlier time in the course of the patient's disease.", 'So this is just illustrating that patient stage presentation and comorbidity of patients, especially among older patients, will also make a difference in outcome.', 'So in summary from some of these papers from the New England Journal of Medicine and every publication in other publications, if one is comparing the post-operative mortality in specialized centers, comparing to community hospitals in the United States there was no difference for variety of tumors involving Breast, Melanoma, Ovarian, Prostate, Thyroid and Uterine.', 'However, there were differences among high risk patients comparing the specialized centers with community hospitals, showing that for the highest risk and high risk patients, there was a significant difference in specializations whereas in patients with low risk there was not a difference between the specialized centers and the community hospitals.', 'And finally, there was a difference for all patients, for these two areas pancreas and esophagus, comparing post-operative mortality. Showing that regardless of the patients risk factors that those patients treated in specialized centers had a significant reduction in post-operative mortality for these two conditions.', 'And here is just one other paper from Dr Birkmeyer in the New England Journal of Medicine, again showing hospital mortality and both based on surgeon volume and hospital volume, showing highly significant differences of relating volume and outcome with regard to thirty day surgically related mortality.', 'The American College of Surgeons has launched on a NSQIP program a national surgical quality improvement program, for which most of the hospitals now in the United States participate in these NSQIP programs and this just shows when a program was initiated at a national level and in this case by the American College of Surgeons, that sixty six percent of the hospitals, now these are almost all the hospitals in the United States, including community hospitals, that sixty six percent of them improve their mortality, eighty two percent improve their morbidity, the worst performing that is low volume surgeons or low volume hospitals, were more likely to improve but even good performers were improved as well.', 'And that the number of low outliers increased and the number of high outliers decrease which is good based upon these quality programs initiated at a national levels.', 'And because of this program, each hospital was estimated to avoid two hundred and fifty complications per year in each hospital on the average.', 'So again I want you to send in your questions by clicking on the Q & A budget and then will resume in another area about the training of oncological surgeons.', "And here's a very important point, although we talk about surgical oncology and super specialisation, the American Board of Surgery, which certifies all general surgeons and general surgery based specialties, include surgical oncology including head and neck surgery among its core areas that are included in its board exams.", 'So surgical oncology and the understanding about the role of surgery in cancer management is equally important to trauma, burns, vascular, pediatrics, critical care, and other criteria.', 'And going beyond that, are two organizations, the Society of Surgical Oncology and the European Society of Surgical Oncology, led by Dr Chandra Are here in the United States, but also including Professor Audisio representing the ESSO, and published two important papers the first showing very large variations in the training of surgical oncologists around the world, and then jointly proposing a global curriculum in surgical oncology that can be used both for training general surgeons but also as curriculum recommendations for training surgical oncology specialists, who are very important at the national level for managing very complex patients and for leading the training programs of general surgeons.', 'And, our curriculum features both in the United States and in these global curriculum included the distinguishing features of surgical oncologists, who manage complex operations, who are part of a multidisciplinary team that involved medical and radiation oncology in treatment planning for patients at the outset of their diagnosis and participate as an oncologist in long term disease management of the cancer patient.', 'And, this recently the American Board of Surgery has approved a new certificate in complex general surgical oncology, acknowledging that there is a sufficient amount of specialized material that justifies a separate sub-board in complex general surgical oncology.', 'Now in a formal level and this comes along the Board certifications in medical oncology, pediatric oncology and radiation oncology there is now a separate Board in complex general surgical oncology.', 'And this two year training involves additional training in multi-disciplinary cancer care, surgical management of oncology conditions, patient counseling on cancer prevention and intervention, and very importantly, the participation and leadership in clinical research and clinical trial design.', 'And this includes such things that actually all general surgeons managing breast cancer for example and especially those surgical oncologists who specialize in breast cancer, need to know how to use ultrasound and core needle biopsy.', 'And I say this because in my travels to other countries there are still many community based centers ,were doing an open biopsy, as to make the diagnosis of breast cancer is the standard of care.', 'Whereas, in the specialty hospitals and in many centres in Europe and in the United States, even at the community level, the standard of care is to use a core needle biopsy instead of an open biopsy, as the preferred treatment for patients with a first diagnosis of breast cancer.', 'Now in these next few slides, I want to emphasize what is a very important part of the surgical patients here in breast cancer in patients who present with stage three disease, that with contemporary chemotherapy and systemic and targeted therapy, patients are now achieving a pathological complete response in forty to sixty percent of such patients, and this is going to change how we manage patients surgically because surgery may not be the first modality of treatment.', 'Here in breast cancer in the United States the majority of breast cancer patients with stage two and stage three breast cancer and I think in many centers in Europe especially for stage three disease, they had systemic therapy is the first treatment and then surgery is the second treatment, so there are many different planning that we have to do with our colleagues in medical oncology about the timing of surgery, the role of the sentinel node which is different in this setting, but this does represent the future of our treatments for many diseases for which breast cancer is the prototype that will change.', "But here's another example, here is a patient with both bulky and unresectable advanced melanoma in the groin and pelvis, who was treated with immunotherapy and after immunotherapy had such a shrinkage that it facilitated the operation, and in fact in this patient when all these masses were removed they were all containing chronic inflammation and scar. This patient had a pathological complete response which could not have been stage without surgical resection of the tumor masses that originally were detected.", 'So the future of cancer systemic therapy is changing dramatically, there are more effective therapies and in fact many of the new therapies are going to be oral therapies that are much less toxicity and will impact in the surgical patient.', 'Here is just an example of where new oral or subcutaneous agents are available for a variety of cancer types.', 'Just to give you some examples, these are the more dramatic examples, but they are getting to be more typical.', 'This is a patient who had extensive metastatic melanoma that even after two weeks of therapy you can see the PET/CT scan showed dramatic reductions in this extensive tumor.', 'And these agents are so powerful, they are already beginning to be applied in stage three melanoma patients.', 'The other major advance that is happening is the use of immunotherapy with the new checkpoint inhibitors, which now have activity in over seventeen different types of cancer, and this does represent our future that will be used increasingly in surgical patients.', 'But just to give you a few individual examples so that you have some idea about the future of therapy, here is an example of a very large melanoma metastasis, that after a single dose of combination immunotherapy this patient had a pathological complete response, demonstrated by surgical excision of the remaining tissue.', "And here's maybe a more famous example, our US President Jimmy Carter, this is in the public arena, presented with liver and brain metastasis. and at age ninety two was treated for a short interval with a single dose of a anti pd1 checkpoint inhibitor.", 'He has now been disease free and in complete remission for over fourteen months, with this single agent in a patient who is much older and tolerated this treatment.', 'I wanted to show yet another example of a patient with head & neck cancer that was ninety six, extensive disease and even after four weeks with this anti pd1 single therapy had a dramatic response.', "So if we're seeing these kind of treatment responses in patients with more advanced disease, you can be sure that this will migrate into the treatment of patients with earlier disease, such as stage two and stage three cancers.", 'So how are we as practicing surgeons going to keep up with these rapidly advancing fields, both fields in systemic therapy, radiation therapy and either in also technological advances in the operating room and new advances in biomarkers used to select patients for these different treatments.', 'And here again, there are a whole variety of ways that this is happening, with live meetings, especially those conducted by the SSO and the ESSO and BASO. But also through affiliations with the tertiary centers such as in my institution, with community hospitals, with advances in journals, with email collaborations or with live webinars, such as this webinars that is being conducted by the European Institute of Oncology.', 'Both Dr Audisio and I, are the editors-in-chief of our two respected journals and we would commend those to you, for keeping up with the field, both the online version and the print version.', 'The Society of Surgical Oncology also now has a self assessment program, that is conducted to help surgeons keep up with the field in oncology.', "And, increasingly we're using telemedicine in the United States, that is increasingly being integrated into the ongoing operations of hospital specialty departments, home health agencies, private physician offices, and in some cases even for educational purposes in the consumer's home.", 'So telemedicine is increasingly being used not only in the United States but around the world, both for medical education to physicians, especially those in the community setting and those in more rural areas, and also for specialist consultative services, especially in helping in patient care decision making, in other locations or even in other nations.', 'The technology allows us now to, with high resolution, to view images of x- rays, images of pathology, and even to do, to look at ultrasound images real time, that are being conducted in another site and transmitted electronically to a central site by telemedicine.', "I've even been involved in a company called Cancer Expert Now, I'm the president of Cancer Expert Now International, where we do consultation with physicians.", 'This shows an example by telemedicine, of one of our neuro oncologists doing a consultation with a physician in another country on a complicated patient with a brain tumor.', 'And, this shows just another example where I was in Huangshan, China. And we conducted a telemedicine virtual tumor board, here with Dr Hope Rugo, who was in San Francisco at the time, discussing a complicated patient with breast cancer.', 'And here is, right after this, a televideo at this, in the same location, on a complex colon cancer patient with Dr Glen Balch who is chief of colorectal surgery at Emory in Atlanta.', 'Again just showing the illustrations about how virtual tumor boards can be used to help in the advice and recommendations for patients with complex cancer, now anywhere in the world.', 'And I just included this slide, for an abstract that was presented last year at ASCO, showing that ninety one percent of virtual tumor board attendees found it to be very helpful, and one hundred percent of the attendees felt that the quality of care was improved.', "Alright and then again we're almost finishing our talk, I'm going to do the summary shortly.", 'And just as a reminder to do your Q & A buttons as I summarize.', 'So how do we as surgeons adapt to a change in a rapidly changing cancer patients?', 'It is clear in the United States and I think in many parts of Europe and elsewhere in the world, that many cancer patients will receive more than one modality of treatment. And that contemporary cancer care is a team approach, combining the collective wisdom of surgeons, medical oncologists, radiation oncologists, and other times with their pathology and radiation oncology.', 'It is clear with the advances in effective systemic therapy, that cancer is becoming a chronic or curable disease.', 'Breast cancer is a very good example of that.', 'And after all of these intensive treatments, survivorships in patients who are living for years after their treatment, are at risk for second primarys, treatment compliance, and the consequences of their intensive multi-disciplinary treatment.', 'The oncology field is one of the most rapidly advancing and changing specialities in all of medicine, and that means that we as surgeons have to keep up through continuing education and incorporate these changes into the curriculum of our trainees.', "And it's also important to validate these advances in surgical patients systematically through clinical trials.", "And as I've tried to emphasize there is a continuing partnerships of professional organizations, especially in the United States, in Europe, on physician continuing education, that has to occur for lifetime. And in improving the curriculum of general surgeons and general surgery based specialities in all, all institutions throughout the world.", 'Surgeons in order to achieve a better outcome, are going to need to partition their patients and identify those, for which specialisation or referral to high volume centres specializing in certain diseases, will have a better outcome for treating patients with complex or advanced cancers.', "And there's both a public and even a government demand for getting optimal care from specialists, multi-disciplinary cancer centres, and regionalization of cancer therapy.", "However, it's important to emphasize, this does not mean that all cancer patients need to be treated by specialists, especially if there's a diagnosis that's made early and the treatment is standard and does not need multiple specialties for their care.", 'We really believe that patients should stay in their local environment, as much as possible, and this is where things like telemedicine may have a role in helping ensure that patients, even in a smaller community hospital, are getting advice and treatment planning, with input from specialists through telemedicine or consultation.', 'And, that those of us who specialize as surgical oncologist, should be educational and research leaders among the general surgery community.', "Now I'm almost finished with my talk, in the summary the professional societies, such as the society I represent, the Society of Surgical Oncology and previously I was CEO of ASCO, the european SSO, the American College of Surgeons, ASCO, are examples of organizations that can be important components of defining the standards for treating surgical patients with cancer and providing education, training, workshops to help other surgeons achieve optimal cancer care.", 'And, that the partnerships across their professional societies, which is well developed between the Europeans in American SSO organizations, will advance our specialty and improve the care for our patients.', 'So that is my conclusion, I will now turn this over to Professor Audisio for his comments and to answer any questions that you may have made during this discussion.', 'Thank you very much.', 'Thank you Charles, we have no words to be grateful to Professor Balch for summarizing the present and the future of good quality cancer surgery worldwide.', 'And I really mean worldwide because this is not just the sophisticated, super advanced, surgical procedure that you might expect from an advanced tertiary referral center, this is a philosophy, this is a very basic attitude, that we should all embrace even in a district general hospital, there is room and there is need for some kind of surgeons, who is aware of the multi-disciplinarity of the oncological practice, and he was keener to talk to different specialties, then offer his own his/her own patients, the most appropriate treatment.', "So am I right to say, Professor Balch, that the bring home message is that good quality cancer surgery is expected, is demanded and it's a worldwide need not just from top cancer units, but from most, from actually all district hospitals as well?", "That's absolutely true and I think the professional organizations and governments in each nation, need to insist that their citizens get high quality care and ensure, in each nation, that there are well trained surgeons, that there are organizational systems, such as regional cancer centers, and I think increasingly that there are relationships internationally with individuals or with organizations, medical centers and cancer centers, that can be of assistance for consultation in complex patients, and in education of the surgeons in the local communities and even in the more provincial or national cancer centers in hospitals.", "For example, next weeky, that I'm doing a multi-disciplinary tumor board with a provincial hospital in China, and part of our theme is going to be, how we can increase our use of televideo in order to increase access for the doctors in China with experts to help in the intellectual decision making for complex cancer patients.", 'I think this is a very hot topic and perhaps we want to spend a little bit more time on how best to facilitate these.', "We've just, you've just point out how the network is there, and I would encourage young surgeons in training to feel absolutely free to follow their dreams, study their topics, and when they meet the big professor at the meeting the pull the sleeve and, and, contact specialist, top specialists, then feel free to communicate with them, perhaps engage with them in the discussion and feel comfortable enough to take advantage of the large network that is available through the SSO, the American Society of Surgical Oncology, and the European one as well, we're at the moment working on facilitating these for developing countries in Africa, through the unrestricted grant that pharmaceutical companies are making available ,this is in my mind, the way forward.", 'So I encourage you guys to come and talk to Professor Balch, to myself, and to whoever you want to improve your knowledge.', "I think the important point is we can't mandate these changes, the initiative has to come from the surgeon in the each community, from their hospital, and from their government, both their local government and their national government, to raise the level of expectations for high quality and timely care.", 'And then work in each nation within the limitations of their resources, in order to achieve that over time.', "Shall we also say words in warning, some easy bite on super advanced technicalities. i.e. You don't want a PET scan, when you don't have a ultrasound machine.", "You might consider not purchasing a robotic equipment, when you're missing a basic laparoscopic stuff, and therefore you need to develop the system, as you're saying, step by step according to the local facilities at a regional or provincial level.", "Yes it's absolutely true, I've been in one major hospital in Africa which has a very expensive radiation therapy machine that is still in boxes because they don't have the radiation physicists or the radiation technicians for the service component, but they have the machine in a box, that somebody bought, but it's more than that.", 'So it takes planning at the government level and at the hospital level, to make sure to define, not just the capital expenditures of buildings and equipment but the delivery of the services by qualified and trained individuals, at all levels.', "So, Professor Balch, we always have enemies and allies, when we want to achieve a target, and that's a bit of a silly and restrictive way to look at it. But it has been the case that most surgical oncologists would blame the general surgeons for not embracing an oncological attitude or philosophy and on the other hand, some medical oncologists are wiping surgeons skills under the carpet.", 'Are we right to say that we actually need everyone and we welcome everyone on board, because this is an ongoing discussion which is just a pleasure to create together as it develops.', 'I agree, and I think the point here is that the surgeon has to be involved in the treatment planning.', "As you and I have talked about there are other specialties and surgeons who spend all their time in the operating room, and allowed the internal medicine trained person to get at the front and make all the decisions and now in that specialty and I'm talking about cardiac surgery, the surgeon is the court of last resort, after all medical treatments have failed.", "And my concern that if the surgeon doesn't bring the surgical perspective to the treatment planning, and is engaged with the medical oncologist and radiation oncologist, the patients may not get the full advantage of a true multi-disciplinary treatment approach that does involve surgery, but we are best trained and have the experience to determine what's the best timing of surgery in some combination and sequence.", 'If we are not at the table, we will be the court of last resort after all else fails with medical treatment and radiation treatment.', 'So we have to be engaged in the treatment planning at the outset.', "Yeah good so once again we totally embrace a multi-disciplinary attitude, where more and more perhaps new figures are taking part for example, I'm thinking of interventional radiologists that only maybe a decade or two decades ago were almost unknown but nowadays there's this stent strictures, they put catheters, they deliver medication and isotopes, and so once again let's straightly reiterate the point that, we want everyone on board here.", "I'm taking advantage of the time because I understand from the organizer that there is a problem with the questions coming in and this has not been made available today, there is a technical hiccup.", "So we do apologise for this and let's make sure that if you have any question you definitely can come back, the audience can definitely come back to me and Professor Balch in writing and we will be pleased to provide any, any answer or satisfy any request.", "But I'm very pleased to be part of this important session and to join you, an internationally renowned surgical oncologist, in this educational forum by televideo.", "I'm also very grateful to the European School of Oncology, cause we often feel that the surgical team is a bit neglected so, I'd like to congratulate ESO for this and perhaps you know, twist their arm and make sure that more space and more interest is devolved towards what surgeons are doing.", 'Is there any conclusive note that you would like to underline Professor Balch?', "Yes, I'd like you to get the last word, I purposely planted the slides about breast cancer management in Great Britain, a high income place.", "And it's my understanding that subsequent to that time around the year 2002-2004, the British government really did engage in cancer center, in regionalization and that the care in breast cancer has greatly improved.", "You live in that arena and you've witnessed that, now wonder if you would just comment because one of our themes is there is a need for some level of specialisation, even in breast cancer.", "Okay, I'm gonna share my thoughts with you and the participants.", "I'm sure that, I hope that I'm not offending anybody, but the story goes that, when I first came to this country in 99, breast surgery was dealt with by the most clumsy surgeons, they were perhaps not very skilled for a visceral surgery, so they were relegated to a subcutaneous cancer surgeon.", 'And therefore, the cosmetic outcomes were unbelievable, and the oncological results were also very, very poor.', "And those days, we were still suffering of the egos, that you know surgeons can do everything, or you don't need, I've cured you, you don't need any chemo, you don't need any radio, you just come back to me in six months, in one year and you're sorted.", 'Now, fortunately, there has been a desire, a decision to establish dedicated breast surgeons.', 'The old guys have retired and now there are more and more young people who are really focusing on breast only.', 'And this means that they are not doing the on call in the night, they are simply focusing on breast cancer surgery.', "Also Professor Audisio, isn't it true that there's been an establishment of cancer centers, on a regional basis that, not only for breast but for other diseases, has elevated the quality of care.", "Yes definitely so, this is particularly true for advanced cancer surgery, as you pointed out, liver surgery, pancreatic surgery, sarcomas, peritoneal disease, there is no point in having twenty units across the country, we have four accredited fully audited, and nicely scrutinised and beautifully performing unit, so that's that's the setting.", 'But the point i was making is that, I think the young generation is now embraced, as you were saying before, the idea of working together and deciding yes surgical issues but at what time? in which patients? how aggressive should we be?', 'And, and what is left after surgery?', 'And, we are definitely improving on this and the results are there, the cancer outcomes have improved.', "I'm not sure that I can quote it due to the, this the establishment of the multi-disciplinary team, but it's definitely true that this cohesive philosophy, this harmonic approach to the cancer patients has definitely milliard improved the the outcomes.", 'Yeah, and I think the teaching point the summary of all of this talk is that, surgeons have to play a role in this rapidly advancing field and that tragically billions of people are not getting access to safe, affordable and timely care.', 'And I think all of us need to redouble our efforts to help cancer patients throughout the world, so that regardless of where they live, they can gain access, either by a more educated and trained local physician, or referral to a regional centre.', 'And that the education process, is enhanced in every country so that we can all keep up with this rapidly growing field and be able to safely introduce these new therapies, as they are approved by governments and insurance schemes.', 'I am very grateful Professor Balch for your time, for your wise vision, and for sharing your thoughts with us.', 'I just want to say you grazie molte, grazie, ciao ciao, arrivederci, grazie Thank you Professor Balch and thank you Professor Audisio.', 'Ladies and gentlemen the session has now concluded.', 'If interested in CME accreditation, participants should fill in the evaluation form and pass the CME multiple choice test.', 'To enter the CME section of the site, please close this window and you will be forwarded to the CME forms.', 'If you are a Mastermind competitor, you will enter the quiz after the evaluation.', 'The next e-ESO session will take place on the twentieth of July at the same time and Professor Stefano Luminari of the University of Modena and Reggio Emilia, Modena, Italy will discuss with Dr Astrid Pavlovsky of the Clinical Research Centre Fundaleu, Buenos Aires, Argentina, the following topic Highlights of ICML 2017.', ' Thank you for participating and have a pleasant evening, day or night good bye.']
,
]